# Import dữ liệu FD Balance mới vào file lưu trữ

In [ ]:
import pandas as pd
import numpy as np
from os import path,getcwd
# import pickle5 as pickle

In [ ]:
def change_type_table(file):
    file.ACCTNO=file.ACCTNO.astype("uint64")
    file.CIFNO=file.CIFNO.astype("uint32")
    file.BRN=file.BRN.astype("uint16")
    file.STATUS=file.STATUS.astype("uint8")
    file.CDTERM=file.CDTERM.astype("uint16")
    file.CDTCOD=file.CDTCOD.astype('category')
    file.RATE=file.RATE.astype('float32')
    file.CURTYP=pd.Categorical(file.CURTYP, categories=['AUD','BND','CAD','CHF','CNY','CZK','DKK','EUR','FJD','GBP','HKD','IDR','INR','JPY','KHR', 'KRW', 'LAK', 'LKR', 'MMK', 'MYR', 'NOK', 'NZD', 'PHP', 'PLN', 'RUB', 'SEK', 'SGD', 'THB', 'TRY', 'TWD', 'USD', 'VND', 'XAU'])

In [ ]:
# path
fd_path=getcwd()
fd_alco=path.join(fd_path,'Alco_31082020')
fd_save =path.join(fd_path,'Python_save')
fd_import_file =r"D:\4. FD-DD behavior\FD\Imported"
# gán biến parameter
end_date=pd.Timestamp('2020-08-31')
start_date=pd.Timestamp('2015-10-31')
list_bank=['L','S','B','C','R',"F","NF"]
list_CCY=['VND','USD','OTH']
tw = [1,7,30,90,180,365]
list_date=pd.date_range(start=start_date,end=end_date).tolist()

In [ ]:
file_data_name=path.join(fd_save,"FD_BALANCE.h5") # tên file lưu trữ là file h5
store=pd.HDFStore(file_data_name)

In [ ]:
# gán biến to import data
to_date=pd.Timestamp('2021-01-24')
store.open()
from_date=store.select_column("data","CHECK_TO_DATE").max()+pd.Timedelta("1D")
list_date_import=pd.date_range(start=from_date,end=to_date).tolist()

In [ ]:
for import_day in list_date_import:
    print(import_day,end='\r')
    store.open()
    file_name="cd_{}".format(import_day.strftime(format="%Y_%m_%d"))
    try:
        data=pd.read_pickle(path.join(fd_import_file,"{}.pickle".format(file_name.upper())))
        data.rename(columns={import_day:"CURRENT_BALANCE"},inplace=True)
    except:
        data=pd.read_sas(path.join(fd_import_file,"{}.sas7bdat".format(file_name)), encoding='iso-8859-1')
        current_balance="CURRENT_BALANCE" if "CURRENT_BALANCE" in data.columns else file_name.upper()
        data.rename(columns={current_balance:"CURRENT_BALANCE"},inplace=True)
    change_type_table(data)
    data["CHECK_TO_DATE_1"]=import_day
    data["CHECK_FROM_DATE_1"]=import_day
    raw=store.select('data','CHECK_TO_DATE==\'{}\''.format((import_day-pd.Timedelta("1D")).strftime(format="%Y-%m-%d")))
    raw=raw.merge(data,how="outer",indicator=True)
    raw.loc[raw._merge!="left_only","CHECK_TO_DATE"]=raw.loc[raw._merge!="left_only","CHECK_TO_DATE_1"]
    raw.loc[raw._merge=="right_only",'CHECK_FROM_DATE']=raw.loc[raw._merge=="right_only",'CHECK_FROM_DATE_1']
    raw.drop(['CHECK_FROM_DATE_1','CHECK_TO_DATE_1',"_merge"],axis=1,inplace=True)
    change_type_table(raw)
    store.remove("data",'CHECK_TO_DATE==\'{}\''.format((import_day-pd.Timedelta("1D")).strftime(format="%Y-%m-%d")))
    store.put('data', raw, format='table', data_columns=True,index=False,append=True)
    store.close()
print("Done!")

# Chạy dữ liệu Timeseries

## Import dữ liệu

In [ ]:
import pandas as pd
import numpy as np
from os import path,getcwd
import os
from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#path
running_tool_path=getcwd()
parameter_path=os.path.join(running_tool_path,'Parameter_data')
parameter_final_export_path=parameter_path+r"\Parameter final 2021\\"

In [ ]:
# currency rate
sfs=pd.read_sas(parameter_path+'\\ssfxrt_2021_02_28.sas7bdat',encoding="utf-8")
sfs.loc[sfs.CURRENCY_CODE=="USD","MID_RATE"]=1 # key lai USD/VND=1

#Import file dữ liệu cbal
store = pd.HDFStore(parameter_path+'\\FD_BALANCE_30062021.h5') # cbal # chú ý ko được để đường dẫn chữ có dấu
#Cbal
store.open()
cbal_raw=store.select("data",auto_close=True).reset_index(drop=True)

In [ ]:
# bank (update testset)
bank_train=pd.read_pickle(parameter_path+'\\bank_full_2021_03_01.pickle')
bank_test=pd.read_pickle(parameter_path+'\\bank_2021_05_02.pickle')
bank_test2=pd.read_pickle(parameter_path+'\\bank_2021_06_30.pickle')
bank_test=bank_test2.append(bank_test).drop_duplicates()
bank_test.loc[(bank_test.CUSTOMER_RATING=="41") & (bank_test.bank=="F"),"bank"]="NF"
bank_test=bank_test.rename(columns={'cif_number':'CIF_NUMBER'})[bank_train.columns]
bank=bank_train.combine_first(bank_test)

bank.to_pickle(parameter_final_export_path+"bank_2021_03_01.pickle")

In [ ]:
# Import dữ liệu xlkt
xlkt_acc=pd.read_excel(parameter_path+'\\xlkt_FD_LR_full_2021_08_31.xlsx',sheet_name='account_xlkt',usecols=[0])
xlkt_cif=pd.read_excel(parameter_path+'\\xlkt_FD_LR_full_2021_08_31.xlsx',sheet_name='cif_xlkt',usecols=[0])

list_xlkt_acc=xlkt_acc.ACCTNO.to_list()
list_xlkt_cif=xlkt_cif.CIFNO.to_list()

#List CD
list_CD=cbal_raw.loc[(cbal_raw.TYPE.str.startswith('CD'))].TYPE.to_list() 

In [ ]:
# xac dinh fcb (lưu ý class_code và class trong file fcb năm tới viết hoa hết để k bỏ sót TH khi merge)
### import dữ liệu fcb 2017_2019
fcb_2017_2019 = pd.read_sas(parameter_path+'\\fcb_2017_2019.sas7bdat',format='sas7bdat',encoding='utf-8')
fcb_2017_2019.rename(columns={'Cif':'CIF','major_class':'CLASS'},inplace=True)
fcb_022021_raw = pd.read_excel(parameter_path+'\\fcb_ 022021.xlsx',sheet_name='Sheet2')
fcb_class_code=pd.read_excel(parameter_path+'\\FCB_class_code.xlsx')

### import dữ liệu fcb 2021
fcb_022021=fcb_022021_raw.merge(fcb_class_code, how='left', on='class').drop(['class'], axis=1).drop_duplicates()
fcb_022021_raw[fcb_022021_raw.duplicated(['CIF'],keep=False)]

### merge fcb
fcb=fcb_022021.append(fcb_2017_2019.loc[(~fcb_2017_2019.CIF.isin(fcb_022021.CIF.to_list()))])

In [ ]:
"""
# import pandas as pd2
# xac dinh incentive
### import dữ liệu khách hàng incentive
incentive_QLTC_2021=pd.read_excel(parameter_path+'\\Incentive FD +Casa Data_202102.xlsx',sheet_name='Incentive FD')[["CUSTOMER_ID","CURRENCY"]].copy()
incentive_QLTC_2021.rename(columns={'CUSTOMER_ID':'CIFNO','CURRENCY':'CURTYP'},inplace=True)

incentive_2016_2019=pd.read_excel(parameter_path+'\\incentive_cd_20162019.xlsx')[['CIFNO','CURTYP']]
incentive_RB_2021=pd.read_csv(parameter_path+'\\Incentive_RB_FD_2020_2021.csv')[['CIFNO','CURTYP']]

incentive=incentive_2016_2019.append(incentive_QLTC_2021).append(incentive_RB_2021)
incentive.drop_duplicates(inplace=True)
incentive.dropna(how='all',inplace=True)

incentive['INCENTIVE']=1
"""

incentive=pd.read_excel(parameter_path+'\\Incentive_full.xlsx')
incentive['INCENTIVE']=1

In [ ]:
#Xử lý số liệu tỷ giá 
## key lai USD/VND=1
sfs.rename(columns={'CURRENCY_CODE':'CURTYP'}, inplace=True)
sfs.loc[sfs.CURTYP.isin(['VND','USD']),'MID_RATE']=1

## Data processing

In [ ]:
#Loại 1 số account, cif xlkt, CD
data=cbal_raw[(cbal_raw.CURRENT_BALANCE>0) 
          & (~cbal_raw.ACCTNO.isin(list_xlkt_acc)) 
          & (~cbal_raw.CIFNO.isin(list_xlkt_cif)) 
          & (~cbal_raw.TYPE.isin(list_CD)) 
         ].copy()

# update the newest CIF for account (có những acc chuyển CIF ) 
data=data.sort_values(["ACCTNO","CHECK_FROM_DATE"]).reset_index(drop=True)
data.loc[data.duplicated(["ACCTNO"],keep="last"),"CIFNO"]=np.nan
data.CIFNO.fillna(method="bfill",inplace=True)

In [ ]:
### set value cho cột incentive cho bảng cbal
data=data.merge(incentive,how='left',on=['CIFNO','CURTYP'])
data.INCENTIVE.fillna(0,inplace=True)

In [ ]:
# quy các đồng tiền khác USD về VND, đổi tên thành "OTHER"
data["CURRENT_BALANCE"]=data[["CURRENT_BALANCE"]].to_numpy()*sfs.set_index("CURTYP").reindex(data.CURTYP).to_numpy()
data.CURTYP=data.CURTYP.astype("str")
data.loc[data.CURTYP.isin(["VND","USD"])==False,"CURTYP"]="OTHER"

In [ ]:
# merge data vs bank
data=data.merge(bank.rename(columns={"CIF_NUMBER":"CIFNO"}),how="left")

In [ ]:
## Vip
data["Time_delta"]=(data["CHECK_TO_DATE"]-data["CHECK_FROM_DATE"])/pd.Timedelta("1D")+1
data["Cbal_Time_delta"]=(data["CURRENT_BALANCE"]*data["Time_delta"])
Danh_muc_avg=data.groupby(["bank","CURTYP"])["Cbal_Time_delta"].agg(np.nansum)/((data.CHECK_TO_DATE.max()-data.CHECK_FROM_DATE.min())/pd.Timedelta("1D"))

cif_sum=data.groupby(["CIFNO","bank","CURTYP"])[["Cbal_Time_delta","Time_delta"]].sum()
cif_avg=(cif_sum.Cbal_Time_delta/cif_sum.Time_delta).reset_index()

res=cif_avg.rename(columns={0:"CIF_avg_cbal"}).merge(Danh_muc_avg.reset_index().rename(columns={0:"danhmuc_avg_cbal"}),how="outer")
res["vip"]=((res.CIF_avg_cbal/res.Cbal_Time_delta)>0.1).astype(int)

data=data.merge(res[['CIFNO','bank','CURTYP','vip']],how='outer').copy()

data["VIP_INCENTIVES"]=0
data.loc[(data["vip"]+data["INCENTIVE"])>0,'VIP_INCENTIVES']=1

data.drop(['vip','INCENTIVE','Cbal_Time_delta','Time_delta'],axis=1,inplace=True)

In [ ]:
### Set giá trị cho fcb or non-fcb
### set class cho khách hàng bank = "R"
data.loc[(data.CIFNO.isin(list(fcb[fcb.CLASS=="G"].CIF))) & (data['bank']=="R"),'CLASS_FCB']="G" 
data.loc[(data.CIFNO.isin(list(fcb[fcb.CLASS=="P"].CIF))) & (data['bank']=="R"),'CLASS_FCB']="P"
data.loc[(data.CIFNO.isin(list(fcb[fcb.CLASS=="D"].CIF))) & (data['bank']=="R"),'CLASS_FCB']="D"
data.loc[(data.CIFNO.isin(list(fcb[fcb.CLASS=="O"].CIF))) & (data['bank']=="R"),'CLASS_FCB']="O" # khách hàng trải nghiệm

In [ ]:
data.to_pickle('Parameter_data/data_handled_30062021.pickle')

## Chạy dữ liệu Timeseries

In [ ]:
import pandas as pd
import numpy as np
from os import path,getcwd
import os
from IPython.display import clear_output
# import pickle5 as pickle
import warnings
warnings.filterwarnings("ignore")


In [ ]:
data_acc_ori=pd.read_pickle("Parameter_data/data_handled_30062021.pickle")

In [ ]:
def modify_bank(data_acc,type_gp=0):
    if type_gp==0:
        return_data=data_acc.copy()
    if type_gp==1:
        return_data=data_acc[data_acc.bank.isin(list("RCLSB"))].copy()
        return_data.bank=return_data.bank.map(lambda x: "RB" if x in "RC" else "EB")
    if type_gp==2:
        return_data=data_acc[data_acc.bank.isin(list("RCLSB"))].copy()
        return_data.bank="BWex"
    if type_gp==3:
        return_data=data_acc.copy()
        return_data.bank="BW"
    if type_gp==4:
        return_data=data_acc[data_acc.bank.isin(list("SB"))].copy()
        return_data.bank="SME"
    if type_gp==5:
        return_data=data_acc[data_acc.bank.isin(list("RC"))].copy()
        return_data.bank="RB_"+data_acc_ori.CLASS_FCB.isna().replace({True:"NONFCB",False:"FCB"})
    return return_data

def data_group(data_acc):
    data_acc["CLASS_FCB"].fillna("N",inplace=True)
    data_acc["KEY_GROUBY"]=data_acc["CURTYP"]+"_"+data_acc["bank"]
    data=data_acc.groupby(["CIFNO","CURTYP","bank","VIP_INCENTIVES","CLASS_FCB","CHECK_FROM_DATE","CHECK_TO_DATE","KEY_GROUBY"],as_index=False)["CURRENT_BALANCE"].sum()
    data["KEY_GROUP_CIF"]=data["CIFNO"].astype(int).astype(str)+"_"+data["KEY_GROUBY"]
    return data_acc,data


def term(data_acc):
    data_acc.loc[(data_acc.CDTCOD=="D") & (data_acc.CDTERM>=360),"TERM_MONTH"]="Long_Term"
    data_acc.loc[(data_acc.CDTCOD=="M") & (data_acc.CDTERM>=12),"TERM_MONTH"]="Long_Term"
    data_acc.TERM_MONTH.fillna("Short_Term",inplace=True)


def run_data_modify(data_acc,type_gp,return_acc=True):
    return_data=modify_bank(data_acc,type_gp=type_gp)
    data_acc,data=data_group(return_data)
    term(data_acc)
    if return_acc:
        return data_acc,data
    else:
        return data

def return_df_tw(table,min_from_date=None,max_from_date=None,min_to_date=None,max_to_date=None,incentive=None,fcb=False,Term_condition=False,KEY_GROUP_CIF=[]): # "Lấy ra dữ liệu theo condition"
    min_from_date=(True if min_from_date==None else (table.CHECK_FROM_DATE>=min_from_date))
    max_from_date=(True if max_from_date==None else (table.CHECK_FROM_DATE<=max_from_date))
    min_to_date=(True if min_to_date==None else (table.CHECK_TO_DATE>=min_to_date))
    max_to_date=(True if max_to_date==None else (table.CHECK_TO_DATE<=max_to_date))
    condi_incentive=(True if incentive==None else ((table.VIP_INCENTIVES==1) if incentive==1 else (table.VIP_INCENTIVES==0)))
    condi_KEY = (True if len(KEY_GROUP_CIF)==0 else (table.KEY_GROUP_CIF.isin(KEY_GROUP_CIF)))
    condi_fcb = (True if fcb==False else (table.CLASS_FCB.isin(["G","P","D","O"])))
    return_table = table[ min_from_date & max_from_date & min_to_date & max_to_date & condi_incentive & condi_fcb & condi_KEY].copy()
    if incentive==1:
        return_table.KEY_GROUBY=return_table.KEY_GROUBY+"_Incentive"
    if incentive==0:
        return_table.KEY_GROUBY=return_table.KEY_GROUBY+"_Treated"
    if fcb:
        return_table.KEY_GROUBY=return_table.KEY_GROUBY+"_"+return_table.CLASS_FCB
    if Term_condition:
        return_table.KEY_GROUBY=return_table.KEY_GROUBY+"_"+return_table.TERM_MONTH
    return return_table

def calculate_MB(date,incentive,fcb,data): 
    date365=date+pd.Timedelta(days=365)
    list_365day=pd.date_range(start=date,end=date365)
    data_at_date=return_df_tw(data,min_to_date=date,max_from_date=date365,incentive=incentive,fcb=fcb
                              ,KEY_GROUP_CIF=return_df_tw(table=data,max_from_date=date,min_to_date=date).KEY_GROUP_CIF.unique()).copy()
    checking=(np.array(data_at_date[["CHECK_FROM_DATE"]])<=(np.array(list_365day))) & (np.array(data_at_date[["CHECK_TO_DATE"]])>=(np.array(list_365day)))
    cbal=np.array(data_at_date[["CURRENT_BALANCE"]])
    res=pd.DataFrame(checking*cbal,index=data_at_date["KEY_GROUBY"])
    res=res.groupby(res.index).sum()
    min_balance=pd.DataFrame([res.values[:,:i+1].min(1) for i in list_tw],columns=res.index,index=list_tw)
    count=data_at_date[["KEY_GROUP_CIF","KEY_GROUBY"]].drop_duplicates("KEY_GROUP_CIF").groupby("KEY_GROUBY").count().rename(columns={"KEY_GROUP_CIF":"count"}).T
    min_balance=count.append(min_balance).T
    min_balance.set_index(pd.MultiIndex.from_tuples([i.split("_",1) for i in min_balance.index],names=["CURTYP","bank"]),inplace=True)
    min_balance = min_balance.melt(ignore_index=False,var_name="Object",value_name=date).set_index("Object",append=True).sort_index().T
    return min_balance

def cal_contractual(date,incentive,fcb,data_acc,tw_list=[7,30,90,180,365]):
    df=return_df_tw(max_from_date=date,min_to_date=date,table=data_acc,incentive=incentive,fcb=fcb).rename(columns={"CURRENT_BALANCE":0})
    for tw in tw_list:
        date_tw=date+pd.Timedelta(days=tw)
        df2=return_df_tw(max_from_date=date_tw,min_to_date=date_tw,table=data_acc,incentive=incentive,fcb=fcb)[["ACCTNO","CURRENT_BALANCE"]]
        df=df.merge(df2.rename(columns={"CURRENT_BALANCE":tw}),how="left")
    li=[]
    for tw in tw_list:
        res=df[(df.OPEN_DATE< date) & (df.MAT_DATE >= date) & (df.MAT_DATE <= (date+pd.Timedelta(days=tw)))].groupby("KEY_GROUBY")[[0,tw]].sum()\
            .rename(columns={0:"contractual{}_0".format(tw),tw:"contractual{}_{}".format(tw,tw)})
        li.append(res.set_index(pd.MultiIndex.from_tuples([i.split("_",1) for i in res.index],names=["CURTYP","bank"]))\
            .melt(ignore_index=False,var_name="Object",value_name=date).set_index("Object",append=True).T)
    return pd.concat(li,axis=1)

def term_count_sum(date,incentive,fcb,data_acc):
    res=return_df_tw(max_from_date=date,min_to_date=date,table=data_acc,incentive=incentive,fcb=fcb,Term_condition=True).groupby("KEY_GROUBY")["CURRENT_BALANCE"].agg({"count","sum"})
    res=res.set_index(pd.MultiIndex.from_tuples([i.split("_",1) for i in res.index],names=["CURTYP","bank"])).reset_index()\
    .melt(id_vars=['CURTYP','bank'],value_vars=["count","sum"],var_name="Object",value_name=date).set_index(['CURTYP','bank','Object'],append=True).T
    return res

def all_port(date,data_acc,data):
    return pd.concat([calculate_MB(date,incentive=None,fcb=False,data=data), # ko xét tới yếu tố incentive, fcb
                      calculate_MB(date,incentive=None,fcb=True,data=data), # ko xét tới yếu tố incentive, fcb phân loại G D P
                      calculate_MB(date,incentive=1,fcb=False,data=data), # danh sách incentive, ko xét fcb
                      calculate_MB(date,incentive=0,fcb=False,data=data), # danh sách non-incentive, ko xét fcb
                      cal_contractual(date,incentive=None,fcb=False,data_acc=data_acc), 
                      cal_contractual(date,incentive=None,fcb=True,data_acc=data_acc),
                      cal_contractual(date,incentive=1,fcb=False,data_acc=data_acc),
                      cal_contractual(date,incentive=0,fcb=False,data_acc=data_acc),
                    term_count_sum(date,incentive=None,fcb=False,data_acc=data_acc)],axis=1)


def run_result_loop(type_gp,list_date,data_acc_ori=data_acc_ori):
#     global Result
    Result=pd.DataFrame()
    data_acc,data=run_data_modify(data_acc_ori,type_gp=type_gp)
    for date in list_date:
        print(date.date(),end="\r")
        if type_gp!=5:
            Result=Result.append(all_port(date,data_acc=data_acc,data=data))
        else:
            Result=Result.append(pd.concat([calculate_MB(date,incentive=None,fcb=False,data=data)
                                            ,calculate_MB(date,incentive=0,fcb=False,data=data)],axis=1))
        clear_output()
    Result.sort_index(axis=1,inplace=True)
    Result.to_pickle("Output_file/Result_part{}.pickle".format(type_gp))

#Count so khach hang theo tung danh muc
def count_cif(data_acc_ori,type_gp):
    data=run_data_modify(data_acc_ori,type_gp,return_acc=False)
    return data.drop_duplicates(["KEY_GROUP_CIF"]).groupby(['CURTYP',"bank","VIP_INCENTIVES","CLASS_FCB"])["CIFNO"].count().sort_index()

### Tính count CIF timeseries

In [ ]:
data_acc_ori_train=data_acc_ori[data_acc_ori.CHECK_FROM_DATE<=pd.Timestamp("2021-02-28")].copy()
data_acc_ori_train.loc[data_acc_ori_train.CHECK_TO_DATE>pd.Timestamp("2021-02-28"),"CHECK_TO_DATE"]=pd.Timestamp("2021-02-28")
data_acc_ori_train=data_acc_ori_train.drop("bank",axis=1).merge(bank.rename(columns={"cif_number":"CIFNO"}),how="left")
count=pd.DataFrame()
for type_gp in range(6):
    count=count.append(pd.DataFrame(count_cif(data_acc_ori_train,type_gp)))
count=count.reset_index()
count.to_excel('Output_file/count_train.xlsx')

### Tính chuỗi dữ liệu 

In [ ]:
%%time
list_tw=[0,1,7,30,90,180,365]
end_date=pd.Timestamp('2021-06-30') # ngày cuối cùng cả train +test model
start_date=pd.Timestamp('2015-10-31')
list_date=pd.date_range(start=start_date,end=end_date).tolist()
for type_gp in [0,1,2,3,4,5]: #change zvalue with equivalent portfolio 
    run_result_loop(type_gp,list_date,data_acc_ori)

In [ ]:
Result_0=pd.read_pickle('Output_file/Result_part0.pickle')
Result_1=pd.read_pickle("Output_file/Result_part1.pickle")
Result_2=pd.read_pickle("Output_file/Result_part2.pickle")
Result_3=pd.read_pickle("Output_file/Result_part3.pickle")
Result_4=pd.read_pickle("Output_file/Result_part4.pickle")
Result_5=pd.read_pickle("Output_file/Result_part5.pickle")
Result_group=pd.concat([Result_0,Result_1,Result_2,Result_3,Result_4,Result_5],axis=1)
Result_group.to_pickle(r'Output_file\Result_group.pickle')

In [ ]:
Result_group=pd.read_pickle(r'Output_file\Result_group.pickle')

### Thống kê top_wd

In [ ]:
def top_wd(date,incentive,fcb,data,min_vnd_wd=50000000000,min_usd_wd=1000000): 
    date1=date+pd.Timedelta(days=1)
    list_1day=pd.date_range(start=date,end=date1)
    data_at_date=return_df_tw(data,min_to_date=date,max_from_date=date1,incentive=incentive,fcb=fcb
                              ,KEY_GROUP_CIF=return_df_tw(table=data,max_from_date=date,min_to_date=date).KEY_GROUP_CIF.unique()).copy()
    checking=(np.array(data_at_date[["CHECK_FROM_DATE"]])<=(np.array(list_1day))) & (np.array(data_at_date[["CHECK_TO_DATE"]])>=(np.array(list_1day)))
    cbal=np.array(data_at_date[["CURRENT_BALANCE"]])
    res=pd.DataFrame(checking*cbal,index=data_at_date["KEY_GROUP_CIF"])
    res=res.groupby(res.index).sum()
    res["wd"]=res[0]-res[1]
    res.set_index(pd.MultiIndex.from_tuples([i.split("_",2) for i in res.index],names=["CIFNO","CURTYP","bank"]),inplace=True)
    res.reset_index(inplace=True)
    res["wd_day"]=date1
    res=res[((res.CURTYP=="VND") & (res.wd>=min_vnd_wd)) | ((res.CURTYP=="USD") & (res.wd>=min_usd_wd))]
    return res.drop([0,1],axis=1)

def run_top_wd_loop(list_date,data_acc_ori=data_acc_ori):
    Result_top_wd=pd.DataFrame()
    data=run_data_modify(data_acc_ori,type_gp=0,return_acc=False)
    for date in list_date:
        print(date.date(),end="\r")
        Result_top_wd=Result_top_wd.append(top_wd(date,incentive=None,fcb=False,data=data,min_vnd_wd=20000000000,min_usd_wd=1000000))
        clear_output()
    Result_top_wd.CIFNO=Result_top_wd.CIFNO.astype(float)
    df_merge=data[["CIFNO","CURTYP","VIP_INCENTIVES","CLASS_FCB"]].drop_duplicates()
    Result_top_wd=Result_top_wd.merge(df_merge,how="left")
    Result_top_wd["Content"]="Ngày " + Result_top_wd["wd_day"].dt.strftime("%d/%m/%Y") +" CIF "+\
                        Result_top_wd["CIFNO"].astype(str) + Result_top_wd["bank"].map(lambda x: " ({}) ".format(x)) +" rút " +\
                        round(Result_top_wd["wd"]/Result_top_wd["CURTYP"].map(lambda x: 1000000000 if x=="VND" else 1000000),1).astype(str)\
                        +Result_top_wd["CURTYP"].map(lambda x: " tỷ VND" if x=="VND" else " triệu USD")
    Result_top_wd=Result_top_wd[Result_top_wd.wd_day!=(list_date[-1]+pd.Timedelta("1D"))]
    Result_top_wd.to_pickle("Output_file/Result_top_wd.pickle")
    Result_top_wd.to_excel("Output_file/Result_top_wd.xlsx")

In [ ]:
%%time
list_tw=[0,1,7,30,90,180,365]
end_date=pd.Timestamp('2021-05-01') # ngày cuối cùng cả train +test model
start_date=pd.Timestamp('2015-10-31')
list_date=pd.date_range(start=start_date,end=end_date).tolist()
run_top_wd_loop(list_date,data_acc_ori)

# Modeling

## Đọc data

In [ ]:
end_train_day=pd.Timestamp("2021-02-28")

# File Timeseries Result
Result_group=pd.read_pickle(r'Output_file/Result_group.pickle')

# Lấy ra tên các cột để phân tách thành các bảng Min balance, Contractual, Term count
cols=Result_group.columns
term_col=[i for i in cols if "Term" in i[1]]
contractual_col=[i for i in cols if "contractual" in str(i[2])]
MB_col=[i for i in cols if (i not in term_col) & (i not in contractual_col)]

# Tạo bảng Min balance
MB_cbal=Result_group[MB_col]

# Sửa tên cột (Thêm 'sum' vào tên cột)
MB_cbal.columns=pd.MultiIndex.from_frame(pd.DataFrame([[i[0],i[1],("sum_{:03}".format(i[2]) if i[2]!="count" else "count" )] for i in MB_cbal.columns],columns=MB_cbal.columns.names))

#phân tách dữ liệu train+test
MB_cbal_train=MB_cbal.loc[:end_train_day].copy()

In [ ]:
def Nan_to_tw(table,level=2):
    df=table.fillna(0)
    for i in df.columns:
        if (i[level]!="count") :
            if (int(i[level][-3:])>0):
                df.loc[df.index[-int(i[level][-3:]):],i]=np.nan
    return df

MB_cbal=Nan_to_tw(MB_cbal)
MB_cbal_train=Nan_to_tw(MB_cbal_train)

```python
#Set số dòng/cột hiển thị tối đa
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
```

## Thống kê mean balance

In [ ]:
Mean_cbal=MB_cbal_train[[i for i in MB_cbal_train.columns if (i[2]=='sum_000') ]].sort_index(axis=1).replace(0,np.nan).mean()
pd.DataFrame(Mean_cbal).reset_index().to_excel('Output_file/Mean Balance Timeseries.xlsx',index=False)

## Tính Core

In [ ]:
# Khai báo seasonal
Solar=[pd.Timestamp(year=y,month=1,day=1)for y in range(2016,2025)]
Lunar=[pd.to_datetime(i,dayfirst=True) for i in ['8/2/2016', '28/1/2017', '16/2/2018', '5/2/2019', '25/1/2020','12/02/2021','01/02/2022','22/01/2023',"10/02/2024"]]
ss_day_t0=[x for y in [pd.date_range(Solar[i]-pd.Timedelta("45D"),Lunar[i]-pd.Timedelta("30D")) for i in range(len(Solar))] for x in y]
st_msb=pd.date_range("2016-07-01","2016-10-12")

### Tính Core

In [ ]:
def Core_Series(Output):
    NHCD=Output.columns[Output.columns.get_level_values(2)=="sum_000"]
    Output2=Output.copy()
    for i in NHCD:
        t_0=Output[[i]]
        t_t=Output.loc[:,(Output.columns.get_level_values(0)==i[0]) 
                       & (Output.columns.get_level_values(1)==i[1]) 
                       & (Output.columns.get_level_values(2).isin([i[2],"count"])==False)].copy().sort_index(axis=1)
        Core=pd.DataFrame(t_t.values/t_0.values,columns=t_t.columns.set_levels(t_t.columns.levels[2].str.replace("sum","Core"), level=2),index=t_t.index)
        Output2=pd.concat([Output2,Core],axis=1)
    return Nan_to_tw(Output2.fillna(0))

MB_Core_all=Core_Series(Output=MB_cbal)
MB_Core_train=Core_Series(Output=MB_cbal_train)

MB_Core_all.to_pickle('Output_file/MB_Core_all.pickle')

### Tính WD Ratio

In [ ]:
MB_Core_all=pd.read_pickle('Output_file/MB_Core_all.pickle')
Withdraw_Ratio=1-MB_Core_all.loc[:,(MB_Core_all.columns.get_level_values(0)!="OTHER") & 
                                         (MB_Core_all.columns.get_level_values(2).str[:4]=="Core")].sort_index(axis=1).copy()
Withdraw_Ratio.columns.set_levels(Withdraw_Ratio.columns.levels[2].str.replace("Core","WD"), level=2,inplace=True)
Withdraw_Ratio.to_pickle("Output_file/Withdraw_Ratio.pickle")

In [ ]:
Withdraw_Ratio_train=1-MB_Core_train.loc[:,(MB_Core_train.columns.get_level_values(0)!="OTHER") & 
                                         (MB_Core_train.columns.get_level_values(2).str[:4]=="Core")].sort_index(axis=1).copy()
Withdraw_Ratio_train.columns.set_levels(Withdraw_Ratio_train.columns.levels[2].str.replace("Core","WD"), level=2,inplace=True)

### Tính Percentile

In [ ]:
def bucket_redistribution(df):
    df=df.set_index(["Time","CURTYP","bank"])
    df2=df.copy()
    for i in df2.columns[1:]:
        df2[i]=(df[i]-df2.loc[:,:i].drop(i,axis=1).sum(1)).map(lambda x:max(0,x))
    df2.columns=["{}_{}".format(a,b) for a,b in  zip(["WD0"]+df.columns[:-1].tolist(),df.columns)]
    df2["Core"]=1-df2.sum(1)
    return df2.reset_index()

def calculate_percentile(ss_day_t0,per_nm=90,per_ss=90,rank_st=-10,Withdraw_Ratio=Withdraw_Ratio_train):
    normal=Withdraw_Ratio[ (Withdraw_Ratio.index.isin(ss_day_t0))==False].apply(lambda x:np.nanpercentile(x.dropna().iloc[-365:],per_nm))
    seasonal=Withdraw_Ratio[Withdraw_Ratio.index.isin(ss_day_t0)].apply(lambda x:np.nanpercentile(x,per_ss))
    stress=Withdraw_Ratio.apply(lambda x:np.sort(x.dropna())[max(rank_st,-len(np.sort(x.dropna())))])
    
    wd_ratio_percentile=pd.concat([normal,seasonal,stress],axis=1).rename(columns={0:"Normal",1:"Seasonal",2:"Stress"}).reset_index()
    wd_ratio_percentile=wd_ratio_percentile.melt(id_vars=['CURTYP','bank',"Object"],value_vars=['Normal','Seasonal','Stress'],var_name="Time",value_name="Ratio")\
        .pivot_table(values="Ratio",columns=["Object"],index=['Time','CURTYP','bank'])
    # wd_ratio_percentile=wd_ratio_percentile.applymap(lambda x:'{:.1%}'.format(x))
    res=wd_ratio_percentile.reset_index().sort_values(["Time","CURTYP","bank"],ascending=[True,False,True])
    return res.merge(bucket_redistribution(res))

per=calculate_percentile(ss_day_t0,per_nm=90,per_ss=90,rank_st=-10,Withdraw_Ratio=Withdraw_Ratio_train)
# per.to_pickle('Output_file/per_train.pickle')
# per.to_excel('Output_file/per_train.xlsx')

## Pilot table

In [ ]:
from os import listdir
from datetime import datetime

import pandas as pd
import numpy as np
from os import path,getcwd
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.dates as mdates
from scipy.ndimage.filters import gaussian_filter1d
%matplotlib inline

# import pickle5

### Khai báo biến

In [ ]:
end_train_day=pd.Timestamp("2021-02-28")
per=pd.read_pickle('Output_file/per_train.pickle')
MB_Core_all=pd.read_pickle('Output_file/MB_Core_all.pickle')
# Khai báo seasonal
Solar=[pd.Timestamp(year=y,month=1,day=1)for y in range(2016,2025)]
Lunar=[pd.to_datetime(i,dayfirst=True) for i in ['8/2/2016', '28/1/2017', '16/2/2018', '5/2/2019', '25/1/2020','12/02/2021','01/02/2022','22/01/2023',"10/02/2024"]]
ss_day_t0=[x for y in [pd.date_range(Solar[i]-pd.Timedelta("45D"),Lunar[i]-pd.Timedelta("30D")) for i in range(len(Solar))] for x in y]
st_msb=pd.date_range("2016-07-01","2016-10-12")

In [ ]:
# import du lieu contractual
# File Timeseries Result
Result_group=pd.read_pickle(r'Output_file/Result_group.pickle')
# Lấy ra tên các cột Contractual
contractual_Result=Result_group[[i for i in Result_group.columns if "contractual" in str(i[2])]]

### Historical plot

In [ ]:
def historical_plot(c="VND",b="RB",tw=365,Output=MB_Core_all):
    uni,div_uni=("Millions",1000000) if c=="USD" else ("Billions",1000000000)
    
    df=Output[(c,b)][["count","sum_000","sum_{:03}".format(tw),"Core_{:03}".format(tw)]]
    df.columns=["COUNT","BALANCE_0","BALANCE_T","CORE_T"]
    
    b_Treated=b+"_Treated"
    df_Treated=Output[(c,b_Treated)][["count","sum_000","sum_{:03}".format(tw),"Core_{:03}".format(tw)]]
    df_Treated.columns=["COUNT","BALANCE_0","BALANCE_T","CORE_T"]
    
    x = df.index  
    fig, axs = plt.subplots(3,1, figsize=(22, 12), gridspec_kw = { "height_ratios":[1,2,2]},dpi=68)
    ax,ax2,ax3 = axs[0],axs[1],axs[2]
    
    # plot time count cif
    ax.grid(alpha=0.4)
    ax.plot(x,df["COUNT"],label="COUNT CIF",alpha=1,linewidth=2)
    ax.plot(x,df_Treated["COUNT"],label="COUNT CIF TREATED",alpha=1,linewidth=2)
    ax.set_ylim(df_Treated["COUNT"].min()*0.9, df["COUNT"].max()*1.1)
    ax.set_title("SỐ LƯỢNG KHÁCH HÀNG DANH MỤC {} {}".format(b,c))
    ax.legend(loc="upper left",ncol=2)

    for axx,dfx,bx in [(ax2,df,b),(ax3,df_Treated,b_Treated)]:
        # plot balance
        axx.grid(alpha=0.4)
        axx.set_title("GIÁ TRỊ DANH MỤC VÀ TỶ LỆ CORE DANH MỤC {} {}".format(bx,c))
        axx.plot(x,gaussian_filter1d(dfx["BALANCE_0"]/div_uni, sigma=2),label="Giá trị danh mục tại thời điểm t=0",alpha=1,linewidth=1)
        axx.fill_between(x, 0, gaussian_filter1d(dfx["BALANCE_0"]/div_uni, sigma=2), alpha=0.2)
        axx.plot(x,gaussian_filter1d(dfx["BALANCE_T"]/div_uni, sigma=2),label="Giá trị danh mục tại thời điểm t={} ngày".format(tw),alpha=1,linewidth=1)
        axx.fill_between(x, 0, gaussian_filter1d(dfx["BALANCE_T"]/div_uni, sigma=2), alpha=0.2)
        axx.set_ylim(0, dfx[['BALANCE_0','BALANCE_T']].max().max()*2/div_uni)
        axx.set_ylabel(uni)
        # plot core
        axx_2=axx.twinx()
        axx_2.set_ylabel('Tỷ lệ Core (%)')
        axx_2.plot(x,gaussian_filter1d(dfx["CORE_T"]*100, sigma=2),color='black',label="NORMAL CORE",alpha=1,linewidth=1.6)
        axx_2.plot(st_msb,gaussian_filter1d(dfx.loc[x.isin(st_msb),"CORE_T"]*100, sigma=2),'r',label="STRESS CORE",alpha=1,linewidth=2.1)
        axx_2.plot(x,gaussian_filter1d(dfx.loc[x.isin(ss_day_t0),"CORE_T"].reindex(x)*100, sigma=2),color='limegreen',label="STRESS CORE",alpha=1,linewidth=2.1)
        axx_2.set_ylim(0, dfx["CORE_T"].max()*110)
        axx_2.plot(x,[100]*len(x),color="red",linestyle=':',linewidth=1.5,alpha=.75)
#         axx.vlines(x=df.index.max()-pd.Timedelta(days=365+tw), ymin=0, ymax=1e15)
        
        #plot normal period
        end_train_day_tw=end_train_day-pd.Timedelta(days=tw)
        axx.vlines(x=x[(x<=end_train_day_tw) & (x.isin(ss_day_t0)==False)][-365], ymin=0, ymax=dfx[['BALANCE_0','BALANCE_T']].max().max()*2/div_uni,ls="--")
        axx.vlines(x=x[(x<=end_train_day_tw) & (x.isin(ss_day_t0)==False)][-1], ymin=0, ymax=dfx[['BALANCE_0','BALANCE_T']].max().max()*2/div_uni,ls="--")
        axx.annotate(text="Normal Period",xy=(x[(x<=end_train_day_tw) & (x.isin(ss_day_t0)==False)][-200],0))
        
        # legend
        axx.legend(loc="upper left",ncol=2)
        axx_2.legend(loc="upper left",bbox_to_anchor=(0.45,1),ncol=3)
    axx.set_xlabel('Begin date')
    plt.show()
    return fig



In [ ]:
for b,c,tw in [("RB","VND",365),("SME","VND",365),("RB","USD",365),("L","VND",365)]:
    historical_plot(b=b,c=c,tw=tw,Output=MB_Core_all).savefig("Plot_output/historical_plot_{}_{}_{}.png".format(b,c,tw),bbox_inches="tight")

### Comparison Plot

In [ ]:
def compare_plot(til,c="VND",b="RB",Object="count",Output=MB_Core_all,sigma=2):
    uni,div_uni=("",1) if Object=="count" else (("Billions",1000000000) if c=="VND" else ("Millions",1000000))
    b=b if type(b)==list else [b]
    c=c if type(c)==list else [c]
    df=Output.loc[:,Output.columns.get_level_values("CURTYP").isin(c if type(c)==list else [c])
             &Output.columns.get_level_values("bank").isin(b if type(b)==list else [b])
             &Output.columns.get_level_values("Object").isin(Object if type(Object)==list else [Object])
             ]
    x = df.index
    fig, ax = plt.subplots(1, figsize=(6.5, 3),dpi=68)
    ax.set_title(til+" DANH MỤC {} {}".format(", ".join(b),"/".join(c)))
    plt.grid(alpha=0.4) # set gridlines 
    for col in df.columns:
        ax.plot(x,gaussian_filter1d(df[col]/div_uni, sigma=sigma),label="Danh mục {} {}".format(col[1],col[0]),alpha=1,linewidth=2)
    ax.legend(loc="upper left")
    ax.set_ylim(None, df.max().max()*1.2/div_uni)
    ax.set_xlabel('Begin date')
    ax.set_ylabel(uni)
    plt.show()
    return fig

In [ ]:
compare_plot("GIÁ TRỊ",c="VND",b=["L","B","S"],sigma=2)

### Gap change

In [ ]:
per_last_year=pd.read_excel("Parameter_data/FD Percentile 30 2019.xlsx").rename(columns={"CURRENCY_TYPE":"CURTYP"})
per_last_year["Year"]=2019


per_this_year=per[per.bank.isin(["SME","RB_Treated","L"])][['Time','CURTYP','bank',"WD_030"]].replace("RB_Treated","RB")
per_this_year["Year"]=2020


per_gap=per_last_year.append(per_this_year).pivot_table(columns=['CURTYP','bank',"Time","Year"])

per_gap.loc[:,(per_gap.columns.get_level_values("bank")=="L") & (per_gap.columns.get_level_values("Time")!="Stress")]=np.nan

In [ ]:
def gap_change_data(c="VND",b="RB",per_gap=per_gap,Output_add_core=MB_Core_all,contractual_Result=contractual_Result):
    per_gap_nhcd=per_gap[(c,b)].sort_index(axis=1)
    per_gap_nhcd.columns=["{}_{}".format(x[0],x[1]) for x in per_gap_nhcd.columns]
    df=Output_add_core[[(c,b,"sum_000"),(c,b,"sum_030")]].dropna().iloc[-90:].copy()
    df.columns=["Balance_t0","Balance_tw"]
    df["WD_tw"]=df["Balance_t0"]-df["Balance_tw"]
    df[per_gap_nhcd.columns]=df[["Balance_t0"]].values*per_gap_nhcd.values
    for con in ["Normal","Seasonal","Stress"]:
        a=np.sort([i for i in df.columns if con in i])
        df[con+"_gap_change"]=df[a[1]]-df[a[0]]
    if b=="L":
        df[["Normal_gap_change","Seasonal_gap_change"]]=0
        na_col=df.columns[df.isna().all()].tolist().copy()
        for i in na_col:
            df[i].fillna(contractual_Result[(c,b,"contractual30_0")],inplace=True)
    return df

In [ ]:
def plot_gap(c):
    uni,div_uni=("Millions",1000000) if c=="USD" else ("Billions",1000000000)
    gap=(gap_change_data(c=c,b="RB")+gap_change_data(c=c,b="SME")+gap_change_data(c=c,b="L")) if c=="VND" else gap_change_data(c=c,b="RB")

    df=gap.iloc[:,2:9]/div_uni
    x=df.index
    fig, axs = plt.subplots(1, 2, figsize=(22, 4),dpi=68)
    ax,ax2 = axs[0],axs[1]

    ax.set_title("DỰ BÁO TỶ LỆ RÚT TIỀN DANH MỤC BWex {}".format(c))
    ax.grid(alpha=0.4)# set gridlines 
    ax.plot(x,df["WD_tw"],color="gray",label="WD thực tế",linewidth=2,alpha=0.7)
    ax.fill_between(x,0,df["WD_tw"],color="gray",alpha=0.2)
    color=["green","orange","brown"]
    m=-1
    for col in df.columns[1:]:
        m+=1
        ax.plot(x,df[col],"-" if m%2==1 else "--",label="WD {}".format(col.replace("_"," ")),alpha=1,linewidth=1.25,color=color[m//2])
    ax.legend(loc="upper left",ncol=4)
    ax.set_ylim(None, df.max().max()*1.8)
    ax.set_xlabel('Begin date')
    ax.set_ylabel(uni)
    
    df=gap.iloc[:,9:]/div_uni
    ax2.set_title("DỰ BÁO TỶ LỆ RÚT TIỀN THAY ĐỔI ẢNH HƯỞNG ĐẾN GAP DANH MỤC BWex {}".format(c))
    plt.grid(alpha=0.4)
    m=-1
    for col in df.columns:
        m+=1
        ax2.plot(x,df[col],color=color[m],label=col)
    ax2.legend(loc="upper left",ncol=3)
    ax2.set_ylim(None, df.max().max()*1.2+ abs(df.min().min()))
    ax2.set_xlabel('Begin date')
    ax2.set_ylabel(uni)
    plt.show()
    return fig

In [ ]:
for c in ["VND","USD"]:
    plot_gap(c).savefig("Plot_output/gap_30_{}_v2.png".format(c),bbox_inches="tight")

### Backtest percentile

#### Handling data

In [ ]:
def backtest_processing_data(c="VND",b="EB",tw=30,per=per,Output=MB_Core_all):
    per_nhcd=per.loc[(per.bank==b) & (per.CURTYP==c),["Time","WD_{:03}".format(tw)]].set_index("Time").sort_index()
    df=Output[(c,b)][["sum_{:03}".format(i) for i in [0,tw]]]
    df.columns=["T0","TW"]
    df["WD"]=df["T0"]-df["TW"]
    df[per_nhcd.index]=(df["T0"].values*per_nhcd.values).T
    df["WDss"]=(df["WD"]*df.index.isin(ss_day_t0)).replace(0,np.nan)
    df["WDst"]=(df["WD"]*df.index.isin(st_msb )).replace(0,np.nan)
    
    #contractual
    df_contractual=contractual_Result[(c,b,"contractual{}_0".format(tw))].copy()
    df_contractual.columns=["Contractual_WD"]
    df["Contractual_WD"]=np.nan
    df["Contractual_WD"].fillna(df_contractual,inplace=True)
    
    if b=="L":
        df["Normal"]=df["Contractual_WD"]
        df["Seasonal"]=df["Contractual_WD"]
        
    return df.iloc[:-tw]

#### Backtest Timeseries

In [ ]:
def Backtest_Timeseries_percentile(b,c,tw,df):
    uni,div_uni=("Millions",1000000) if c=="USD" else ("Billions",1000000000)
    df=df/div_uni
    if b=="L":
        df=df.drop(["Normal","Seasonal"],axis=1)
    x = df.index
    df_backtest=df.loc[end_train_day-pd.Timedelta(days=tw-1):]
    x_backtest=df_backtest.index
    # plot time count cif
    fig, axs = plt.subplots(2,1, figsize=(18, 10), gridspec_kw = { "height_ratios":[4,1]},dpi=68)
    ax1,ax2 = axs[0],axs[1]
    ax1.set_title("BACKTEST RÚT TIỀN DANH MỤC {} {}".format(b,c))
    ax2.set_title("GIAI ĐOẠN BACKTEST")
    ax1.grid(alpha=0.4)
    cols=["WD","WDss","WDst","Normal","Seasonal","Stress"]
    col_labels=["Giá trị rút tại tại {} ngày sau tại thời điểm T0".format(tw),
               "Giá trị rút tại tại {} ngày sau tại thời điểm T0 là giai đoạn mùa vụ".format(tw),
                "Giá trị rút tại tại {} ngày sau tại thời điểm T0 là giai đoạn căng thẳng".format(tw),
                "Ước lượng rút thông thường","Ước lượng rút mùa vụ","Ước lượng rút căng thẳng"
               ]
    Colors=["grey","green","orange","brown","blue","red"]
    for co,la,color in zip(cols,col_labels,Colors):
        try:
            ax1.plot(x,gaussian_filter1d(df[co], sigma=1),label=la,color=color)
            ax2.plot(x_backtest,gaussian_filter1d(df_backtest[co], sigma=1),color=color)
        except:
            print("lỗi ",co)
            pass
    ax1.fill_between(x, 0, gaussian_filter1d(df["WD"], sigma=1), alpha=0.2)
    ax2.fill_between(x_backtest, 0, gaussian_filter1d(df_backtest["WD"], sigma=1), alpha=0.2)

    
    # plot contractual
    if b!="L":
        ax1_twin=ax1.twinx()
        ax1_twin.plot(x,gaussian_filter1d(df["Contractual_WD"], sigma=1),label="Giá trị rút theo hợp đồng trong time-window",color="deepskyblue",alpha=0.8)
        ax1_twin.fill_between(x, 0, gaussian_filter1d(df["Contractual_WD"], sigma=1), alpha=0.2,color="deepskyblue")
        ax1_twin.legend(loc="upper right")
        ax1_twin.set_ylabel(uni+" (Contractual)")
        ax1_twin.set_ylim(0, df["Contractual_WD"].max()*1.3)
    else:
        ax1.plot(x,gaussian_filter1d(df["Contractual_WD"], sigma=1),label="Giá trị rút theo hợp đồng trong time-window",color="deepskyblue",alpha=1)

    
    #plot normal period
    end_train_day_tw=end_train_day-pd.Timedelta(days=tw)
    ax1.vlines(x=x[(x<=end_train_day_tw) & (x.isin(ss_day_t0)==False)][-365], ymin=0, ymax=df['WD'].max()*5,ls="--")
    ax1.vlines(x=x[(x<=end_train_day_tw) & (x.isin(ss_day_t0)==False)][-1], ymin=0, ymax=df['WD'].max()*5,ls="--")
    ax1.annotate(text="Normal Period",xy=(x[x.isin(ss_day_t0)==False][-200],df['Stress'].max()*1.1))
    
    ax1.legend(loc="upper left",ncol=2)
    ax1.set_ylim(0, df[['Stress']].max().max()*1.3)
    ax2.set_ylim( df_backtest["WD"].min()*0.9, df_backtest["WD"].max()*1.1)
    ax1.set_ylabel(uni)
    ax2.set_ylabel(uni)
    ax1.set_xlabel('Begin date')
    
    plt.grid(alpha=0.4)
    plt.show()
    return fig

In [ ]:
for b,c,tw in [("RB","VND",30),("SME","VND",30),("RB_Treated","USD",30),("L","VND",30),("BWex","VND",30),("RB","VND",365)]:
    if b=="BWex":
        df=backtest_processing_data(c=c,b="RB",tw=tw,per=per,Output=MB_Core_all)\
        +backtest_processing_data(c=c,b="SME",tw=tw,per=per,Output=MB_Core_all)\
        +backtest_processing_data(c=c,b="L",tw=tw,per=per,Output=MB_Core_all)
    else:  
        df=backtest_processing_data(c=c,b=b,tw=tw,per=per,Output=MB_Core_all)
    Backtest_Timeseries_percentile(b=b,c=c,tw=tw,df=df)#.savefig("Plot_output/Backtest_Timeseries_percentile_{}_{}_{}.png".format(b,c,tw),bbox_inches="tight")

#### Backtest Violation

In [ ]:
def plot_backtest_violation(df,b,c,tw):
    df=df[df.index<=(end_train_day-pd.Timedelta(days=tw))].copy()
    uni,div_uni=("Millions",1000000) if c=="USD" else ("Billions",1000000000)
    list_fig=[]
    list_condition=["Stress"] if b=="L" else ["Normal","Seasonal","Stress"]
    for condition in list_condition:
        fig, axs = plt.subplots(1, 2, figsize=(22.5, 3),dpi=68)
        if condition=="Normal":
            df2=df[df.index.isin(ss_day_t0)==False].iloc[-365:].copy()
        elif condition=="Seasonal":
            df2=df[df.index.isin(ss_day_t0)].copy()
        else:
            df2 = df.copy()
        df2["number_vio"]=(df2["WD"]>df2[condition]).astype(int)
        df2["amount_vio"]=(df2["WD"]-df2[condition])*df2["number_vio"]
        df2["per_amount_vio"]=df2["amount_vio"]/df2["T0"]
#         global df2_month
        df2_month=df2.groupby(pd.Grouper(freq='MS')).agg({"number_vio":"sum","amount_vio":"mean","per_amount_vio":["max","mean"]}).applymap(lambda x: x if x>0 else np.nan)
        df2_month.columns=df2_month.columns.map(lambda x: x[0]+"_"+x[1])
        x=df2_month.index
        
        ax_1,ax_2 = axs[0],axs[1]
        #count violation
        if condition=="Normal":
            ax_1.set_xticks(x,minor=True)
        ax_1.grid(alpha=0.4)
        ax_1.bar(x,df2_month.number_vio_sum.fillna(0),width=15)
        ax_1.set_ylabel('Số vi phạm')
        ax_1.set_title("BACKTEST SỐ LẦN VI PHẠM DANH MỤC {} {} ({})".format(b,c,condition))
        for i in df2_month.dropna().index:
            ax_1.annotate(text=df2_month.loc[i,"number_vio_sum"].astype(int),xy=(i,df2_month.loc[i,"number_vio_sum"]),  va='center',xytext=(-3,7), textcoords='offset points')

        dayloc = mdates.MonthLocator([3,6,9,12])
        ax_1.xaxis.set_major_locator(dayloc)
        ax_1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        try:
            ax_1.set_ylim(0, df2_month.number_vio_sum.max()*1.2)
        except:
            pass
        ax_1.set_xlabel('Begin date')
        
        # balance of violation
        if condition=="Normal":
            ax_2.set_xticks(x,minor=True)
        ax_2.set_title("BACKTEST MỨC ĐỘ VI PHẠM DANH MỤC {} {} ({})".format(b,c,condition))
        ax_2.grid(alpha=0.4)
        ax_2.fill_between(x, 0, df2_month['per_amount_vio_max'].fillna(0)*100, alpha=0.4,color="orange",label="Tỷ lệ (%) vi phạm lớn nhất")
        ax_2.fill_between(x, 0, df2_month['per_amount_vio_mean'].fillna(0)*100, alpha=1,color="orange",label="Tỷ lệ (%) vi phạm trung bình")
        ax_2.set_ylabel('%')
        ax_2.set_xlabel('Begin date')
        
        ax2_2=ax_2.twinx()
        ax2_2.bar(x,df2_month["amount_vio_mean"].fillna(0)/div_uni,width=15,label="Giá trị vi phạm trung bình")
        ax2_2.set_ylabel(uni)
            
        ax_2.legend(loc="upper left",ncol=2)
        ax2_2.legend(loc="upper right")
    
        ax_2.xaxis.set_major_locator(dayloc)
        ax_2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        try:
            ax_2.set_ylim(0, df2_month['per_amount_vio_max'].max()*140)
            ax2_2.set_ylim(0, df2_month['amount_vio_mean'].max()/div_uni*1.4)
        except:
            pass
        fig.autofmt_xdate()
        plt.show()
        list_fig.append(fig)
    return list_fig,list_condition

In [ ]:
for b,c,tw in [("RB","VND",30),("SME","VND",30),("L","VND",30),("RB_Treated","USD",30),("BWex","VND",30),("RB","VND",365)]:
    if b=="BWex":
        df=backtest_processing_data(c=c,b="RB",tw=tw,per=per,Output=MB_Core_all)\
        +backtest_processing_data(c=c,b="SME",tw=tw,per=per,Output=MB_Core_all)\
        +backtest_processing_data(c=c,b="L",tw=tw,per=per,Output=MB_Core_all)
    else:  
        df=backtest_processing_data(c=c,b=b,tw=tw,per=per,Output=MB_Core_all)
    fig,list_condition=plot_backtest_violation(df=df,b=b,c=c,tw=tw)
    for i,con in zip(fig,list_condition):
        i#.savefig("Plot_output/plot_backtest_violation_{}_{}_{}_{}.png".format(b,c,tw,con),bbox_inches="tight")

### Term-count plot

In [ ]:
# import du lieu contractual
# File Timeseries Result
Result_group=pd.read_pickle(r'Output_file/Result_group.pickle')
#Percent long term deposit (>=12M) in total portfolio
Term_count_sum=Result_group[[i for i in Result_group.columns if  "Term" in i[1]]]

In [ ]:
def Term_plot(c='VND',b='RB',Term_count_sum=Term_count_sum):
    
    df_long=Term_count_sum[c,b+'_Long_Term'][['count','sum']]
    df_long.columns=['count_long','sum_long']
    df_short=Term_count_sum[c,b+'_Short_Term'][['count','sum']]
    df_short.columns=['count_short','sum_short']
    df=pd.concat([df_long,df_short],axis=1)
    df['% No. of account']=df['count_long']/(df['count_long']+df['count_short'])
    df['% Balance']=df['sum_long']/(df['sum_long']+df['sum_short'])
    x = df.index
    
    # plot time count cif
    fig, ax = plt.subplots(1, figsize=(20, 2.5),dpi=68)
    plt.grid(alpha=0.4) # set gridlines 
#     plt.grid(alpha=0,axis="y")
    ax.plot(x,df["% No. of account"]*100,label="% No. of account",alpha=1,linewidth=2)
    ax.plot(x,df["% Balance"]*100,label="% Balance",alpha=1,linewidth=2)
    ax.set_title("Percent of long term deposit in total portfolio {} {}".format(b,c))
    ax.set_ylabel('%')
    # legend
    ax.legend(loc="upper right")
    plt.show()
    return fig

In [ ]:
for c,b in [("VND","RB"),("VND","SME"),("USD","RB")]:
    Term_plot(c=c,b=b)#.savefig("Plot_output/Term_plot_{}_{}_{}.png".format(b,c,"long"),bbox_inches="tight")

## Hồi quy

In [ ]:
import pandas as pd
from os import path,getcwd
import os
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.dates as mdates

# from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from scipy.ndimage.filters import gaussian_filter1d
import warnings
# import ast
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

warnings.filterwarnings("ignore")

### Khai báo biến

In [ ]:
#path
running_tool_path=getcwd()
parameter_path=os.path.join(running_tool_path,'Parameter_data')

# Khai báo seasonal
Solar=[pd.Timestamp(year=y,month=1,day=1)for y in range(2016,2025)]
Lunar=[pd.to_datetime(i,dayfirst=True) for i in ['8/2/2016', '28/1/2017', '16/2/2018', '5/2/2019', '25/1/2020','12/02/2021','01/02/2022','22/01/2023',"10/02/2024"]]
ss_day_t0=[x for y in [pd.date_range(Solar[i]-pd.Timedelta("45D"),Lunar[i]-pd.Timedelta("30D")) for i in range(len(Solar))] for x in y]
st_msb=pd.date_range("2016-07-01","2016-10-12")

# kết quả percentile ==> tạo kết quả mix
per=per_gap=pd.read_pickle('Output_file/per_train.pickle')
MB_Core_all=pd.read_pickle('Output_file/MB_Core_all.pickle')

# import du lieu contractual
# File Timeseries Result
Result_group=pd.read_pickle(r'Output_file/Result_group.pickle')
# Lấy ra tên các cột Contractual
contractual_Result=Result_group[[i for i in Result_group.columns if "contractual" in str(i[2])]]

### Xử lý dữ liệu

#### Biến ngày

In [ ]:
start_day_reg=pd.Timestamp("2018-07-02") # Ngày đầu tiên chạy hồi quy
end_day_reg=pd.Timestamp("2021-06-30")
end_day_reg_train=end_train_day=pd.Timestamp("2021-02-28")
#số ngày giai đoạn test
test_size=int((end_day_reg-end_day_reg_train)/pd.Timedelta("1D"))

#covid
covid=pd.date_range(pd.Timestamp("2020-02-14"),end_day_reg).to_list()

# Khai báo seasonal
Solar=[pd.Timestamp(year=y,month=1,day=1)for y in range(2016,2025)]
Lunar=[pd.to_datetime(i,dayfirst=True) for i in ['8/2/2016', '28/1/2017', '16/2/2018', '5/2/2019', '25/1/2020','12/02/2021','01/02/2022','22/01/2023',"10/02/2024"]]
ss_day_t0=[x for y in [pd.date_range(Solar[i]-pd.Timedelta("45D"),Lunar[i]-pd.Timedelta("30D")) for i in range(len(Solar))] for x in y]


#### Biến phụ thuộc

In [ ]:
# Biến tỷ lệ rút (biến phụ thuộc)
Withdraw_Ratio=pd.read_pickle('Output_file/Withdraw_Ratio.pickle')
wd_reg=Withdraw_Ratio.loc[(Withdraw_Ratio.index>=start_day_reg) & (Withdraw_Ratio.index<=end_day_reg),
((Withdraw_Ratio.columns.get_level_values("CURTYP")=="VND") & (Withdraw_Ratio.columns.get_level_values("bank").isin(["RB","SME","EB"]))) |\
((Withdraw_Ratio.columns.get_level_values("CURTYP")=="USD") & (Withdraw_Ratio.columns.get_level_values("bank").isin(["RB_Treated"])))]
wd_reg.columns=["_".join(i) for i in wd_reg.columns]

#### Biến tỷ lệ rút trung binh lag

In [ ]:
# Biến tỷ lệ rút trung binh lag 7 15 30 ngay (biến doc lap)
def avg_wd_lag(lag,Withdraw_Ratio):
    Withdraw_Ratio_copy=Withdraw_Ratio.loc[:,
    ((Withdraw_Ratio.columns.get_level_values("CURTYP")=="VND") & (Withdraw_Ratio.columns.get_level_values("bank").isin(["RB","SME","EB"]))) |\
    ((Withdraw_Ratio.columns.get_level_values("CURTYP")=="USD") & (Withdraw_Ratio.columns.get_level_values("bank").isin(["RB_Treated"])))].copy()
    Withdraw_Ratio_copy.columns=["_".join(i)+"_avg_lag{}".format(lag) for i in Withdraw_Ratio_copy.columns]
    wd_reg_lag=pd.DataFrame(index=Withdraw_Ratio_copy.index,columns=Withdraw_Ratio_copy.columns).fillna(0)
    for i in range(lag):
        wd_reg_lag=wd_reg_lag.add(Withdraw_Ratio_copy.shift(1+i),fill_value=0)
    wd_reg_lag=wd_reg_lag.loc[(Withdraw_Ratio.index>=start_day_reg) & (Withdraw_Ratio.index<=end_day_reg)]/lag
    return wd_reg_lag
    
avg_wd_lag7=avg_wd_lag(lag=7,Withdraw_Ratio=Withdraw_Ratio)    
avg_wd_lag15=avg_wd_lag(lag=15,Withdraw_Ratio=Withdraw_Ratio) 
avg_wd_lag30=avg_wd_lag(lag=30,Withdraw_Ratio=Withdraw_Ratio)
# Avg_wd_lag_variables=pd.concat([avg_wd_lag7,avg_wd_lag15,avg_wd_lag30],axis=1)

In [ ]:
# Biến tỷ lệ rút trung binh và lastday lag cuối tháng (biến doc lap)
def wd_lag_mark_lastmonth(Withdraw_Ratio,func="mean"):
    Withdraw_Ratio_copy=Withdraw_Ratio.loc[:,
    ((Withdraw_Ratio.columns.get_level_values("CURTYP")=="VND") & (Withdraw_Ratio.columns.get_level_values("bank").isin(["RB","SME","EB"]))) |\
    ((Withdraw_Ratio.columns.get_level_values("CURTYP")=="USD") & (Withdraw_Ratio.columns.get_level_values("bank").isin(["RB_Treated"])))].copy()
    Withdraw_Ratio_copy.columns=["_".join(i)+"_{}_lastmonth".format(func) for i in Withdraw_Ratio_copy.columns]
    Withdraw_Ratio_groupby=Withdraw_Ratio_copy.groupby(pd.Grouper(freq='M')).agg(func).reindex(Withdraw_Ratio_copy.index).sort_index()
    Withdraw_Ratio_groupby_available=Withdraw_Ratio_groupby.apply(lambda i: i.shift(int(i.name.split("_")[-3])+1))
    Last_wd_lag_mark_lastmonth_variables_get_beginnextmonth=Withdraw_Ratio_groupby_available.groupby(pd.Grouper(freq='M')).agg("last").reindex(Withdraw_Ratio_copy.index).sort_index().shift(1).ffill()
    return Last_wd_lag_mark_lastmonth_variables_get_beginnextmonth.loc[(Withdraw_Ratio.index>=start_day_reg) & (Withdraw_Ratio.index<=end_day_reg)]

Avg_wd_lag_mark_lastmonth_variables=wd_lag_mark_lastmonth(Withdraw_Ratio,func="mean")
Last_wd_lag_mark_lastmonth_variables=wd_lag_mark_lastmonth(Withdraw_Ratio,func="last")

#### Biến vĩ mô GDP CPI

In [ ]:
# Biến vĩ mô GDP CPI
GDP=pd.read_excel(parameter_path+'\\Macro_variables.xlsx',sheet_name='GDP') #Tỷ lệ tăng trưởng GDP  so với cùng kỳ năm ngoái (thu thập dữ liệu theo quý)
CPI=pd.read_excel(parameter_path+'\\Macro_variables.xlsx',sheet_name='CPI') #Tỷ lệ tăng trưởng CPI  so tháng trước (thu thập dữ liệu theo tháng)
Macro_variables=GDP.merge(CPI).set_index("Date")
Macro_variables=Macro_variables.reindex(pd.date_range(min(Macro_variables.index.min(),wd_reg.index.min()),
                                      max(Macro_variables.index.max(),wd_reg.index.max()))
                       ).ffill().reindex(wd_reg.index)

#### Biến contractual

In [ ]:
# Biến contractual
Result_group=pd.read_pickle(r'Output_file/Result_group.pickle')
Result_group_reg=Result_group.iloc[(Withdraw_Ratio.index>=start_day_reg) & (Withdraw_Ratio.index<=end_day_reg),
((Result_group.columns.get_level_values("CURTYP")=="VND") & (Result_group.columns.get_level_values("bank").isin(["RB","SME","EB","BWex"]))) |\
((Result_group.columns.get_level_values("CURTYP")=="USD") & (Result_group.columns.get_level_values("bank").isin(["RB_Treated"])))]
contractual=Result_group_reg[[i for i in Result_group_reg.columns if ((('contractual' in str(i[2])) & ('_0' in str(i[2]))) or i[2]==0)]].copy().sort_index(axis=1)
contract_rate_variables=pd.DataFrame(columns=contractual.columns[contractual.columns.get_level_values(2).str.contains("contractual").fillna(False)]
                                  ,index=contractual.index)
for i in contract_rate_variables.columns:
    contract_rate_variables[i]=contractual[i].values/contractual[(i[0],i[1],0)].values
contract_rate_variables.sort_index(axis=1,inplace=True)
# contract_rate_variables.columns=pd.MultiIndex.from_tuples([(i[0],i[1],i[2].replace("contractual","contractual_ratio_").replace("_0","")) 
#                                                            for i in contract_rate_variables.columns],names=contract_rate_variables.columns.names)
contract_rate_variables.columns=contract_rate_variables.columns.map(lambda i:"{}_{}_{}".format(i[0],i[1],i[2].replace("contractual","contractual_ratio_").replace("_0","")))

#### Biến lãi suất

```python
# Xử lý dữ liệu thô lãi suất : file IR Survey.xlsx
st_date="2018-07-02"
end_date="2021-01-24"
# XỬ LÝ IR
IR=pd.read_excel(path.join(fd_save,"IR Survey.xlsx"),sheet_name="RAW")
# drop duplicated
IR_hanlded=pd.DataFrame()
for i in IR.BANK.unique():
    IR_bank=IR[IR.BANK==i].copy().ffill().drop_duplicates(IR.columns[:-1])
    IR_bank.loc[IR_bank.Date.duplicated(),"Date"]=IR_bank.loc[IR_bank.Date.duplicated(),"Report_Date"]
    IR_hanlded=IR_hanlded.append(IR_bank)
# pivot_table to fill NaN
IR=IR_hanlded.pivot_table(values=IR_hanlded.columns[5:-1],columns="BANK",index="Date").reorder_levels([1,0],axis=1).sort_index(axis=1)
IR=IR.reindex(pd.date_range(IR.index.min(),IR.index.max())).ffill().reindex(pd.date_range(st_date,end_date)).dropna(how="all").reset_index()
# Melt table
IR=IR.melt("index",var_name=["Bank","Term"],value_name="rate").pivot_table("rate",["index","Bank"],"Term")
# Interpolate missing value (tpbank 15M)
IR.columns=IR.columns.map(lambda x:int(x[:-1]))
IR=IR.sort_index(axis=1).reset_index().rename(columns={"index":"day"})
# set group bank 
G1=["ACB","EXIMBANK","MB","SACOMBANK","TECHCOMBANK","VIB"]
G2=["LIENVIET BANK","DONG A BANK","PVCOMBANK","SCB","SHB","SEABANK","TP BANK","VP BANK"]
IR["group"]=IR.Bank.map(lambda x: "G1" if x.upper() in G1 else ("G2" if x.upper() in G2 else "MSB"))
# groupby by Date and Group
IR_groupby_term=IR.groupby(["day","group"],as_index=False).agg(np.nanmean)
IR_groupby_term.columns=IR_groupby_term.columns.map(lambda x: x if type(x) == str else str(x)+"M")
IR_groupby_term.to_pickle(path.join(fd_save,"IR_groupby_term.pickle"))
```

In [ ]:
IR_groupby_term=pd.read_pickle(r'Output_file\\IR_groupby_term.pickle')

In [ ]:
# Bien lai suat
IR_groupby_term=pd.read_pickle(r'Output_file\\IR_groupby_term.pickle')
select_term=["1M", '6M','12M',"Short_Term","Medium_Term","Long_Term"]
# Group Term LS
IR_groupby_term["Short_Term"]=(IR_groupby_term["1M"]+IR_groupby_term["3M"])/2
IR_groupby_term["Medium_Term"]=(IR_groupby_term["6M"]+IR_groupby_term["9M"])/2
IR_groupby_term["Long_Term"]=(IR_groupby_term["12M"]+IR_groupby_term["13M"]+IR_groupby_term["15M"]+
                              IR_groupby_term["18M"]+IR_groupby_term["24M"]+IR_groupby_term["36M"])/6

G1=IR_groupby_term[IR_groupby_term.group=="G1"].set_index("day").drop("group",axis=1)[select_term]
G2=IR_groupby_term[IR_groupby_term.group=="G2"].set_index("day").drop("group",axis=1)[select_term]
MSB=IR_groupby_term[IR_groupby_term.group=="MSB"].set_index("day").drop("group",axis=1)[select_term]

MSB_G1=MSB-G1
MSB_G2=MSB-G2
MSB.columns="MSB_"+MSB.columns
MSB_G1.columns="MSB_G1_"+MSB_G1.columns
MSB_G2.columns="MSB_G2_"+MSB_G2.columns

IR_variable=pd.concat([MSB,MSB_G1,MSB_G2],axis=1)
IR_variable.index.name="Date"
IR_variable=IR_variable.reindex(wd_reg.index).ffill()

#### biến dummy condition period

In [ ]:
# biến dummy condition period
dummy_period_variables=pd.DataFrame(columns=["ss_dummy","covid_dummy"],index=wd_reg.index)
dummy_period_variables["ss_dummy"]=dummy_period_variables.index.isin(ss_day_t0).astype(int)
dummy_period_variables["covid_dummy"]=dummy_period_variables.index.isin(covid).astype(int)

#### Gộp biến 

In [ ]:
data_all=pd.concat([wd_reg,
                    avg_wd_lag7,avg_wd_lag15,avg_wd_lag30,
                    Avg_wd_lag_mark_lastmonth_variables,Last_wd_lag_mark_lastmonth_variables,
                    Macro_variables,
                    contract_rate_variables,
                    IR_variable,
                    dummy_period_variables],axis=1)

In [ ]:
data_all.to_excel("Regression/data_all.xlsx")

### Phân tích hồi quy

#### Function

In [ ]:
def add_IR_delta(IR_variable,tw):
    df=IR_variable-IR_variable.shift(7) # fix so sánh với 7 ngày trước
    df.bfill(0,inplace=True)
    df.columns=[i+"_delta{}".format(tw)for i in df.columns]
    return df

In [ ]:
def filter_IR_with_correlation(IR_variable,test_size=0,max_corr=0.5):
    IR_corr_matrix=IR_variable.iloc[:(None if test_size==0 else -test_size)].corr()
    # lựa chọn tập X trên threshold correlation
    sign=(IR_corr_matrix.abs()<max_corr).astype(int)
    list_variables=IR_variable.columns.tolist()
    list_X_feature=[[i] for i in list_variables]
    for n in range(2, len(list_variables)+1):
        check_n_1=[i for i in list_X_feature if len(i)==(n-1)]
        if len(check_n_1)==0:
            break
        list_X_feature_n=[]
        for a in check_n_1:
            list_X_feature_n+=[np.sort(a+[x]).tolist() for x in sign[sign[a].prod(1)==1].index]
        list_X_feature+=pd.DataFrame(list_X_feature_n).drop_duplicates().values.tolist()
    return list_X_feature
IR_list=[None]+filter_IR_with_correlation(IR_variable,test_size=0,max_corr=0.5)

In [ ]:
def handling_data_plot(df,tw,c="VND",b="RB",per=None,test_size=test_size,type_adjust=["max"]):
    df=df.copy()
    x=df.index
    x_end_train=x[-test_size]
    df.columns=["Real","Predict"]
    
    # Cách hiệu chỉnh
    for type_adjust in ["max","normal","seasonal"]:
        if type_adjust=="max":
            adjust_reg=(df["Real"]-df["Predict"]).loc[:x_end_train-pd.Timedelta("1D")].max()
            df["Predict_adjust_{}".format(type_adjust)]=df["Predict"]+adjust_reg
            print("Giá trị hiệu chỉnh {} (%) = ".format(type_adjust),round(adjust_reg,2))
        if type_adjust=="normal":
            percentile_adjust=90
            adjust_reg=np.nanpercentile((df["Real"]-df["Predict"]).loc[df.index.isin(ss_day_t0)==False].dropna().loc[:x_end_train-pd.Timedelta("1D")].iloc[-365:],percentile_adjust)
            df["Predict_adjust_{}".format(type_adjust)]=df["Predict"]+adjust_reg
            print("Giá trị hiệu chỉnh {} (%) = ".format(type_adjust),round(adjust_reg,2))
        if type_adjust=="seasonal":
            percentile_adjust=90
            adjust_reg=np.nanpercentile((df["Real"]-df["Predict"]).loc[df.index.isin(ss_day_t0)].dropna().loc[:x_end_train-pd.Timedelta("1D")],percentile_adjust)
            df["Predict_adjust_{}".format(type_adjust)]=df["Predict"]+adjust_reg
            print("Giá trị hiệu chỉnh {} (%) = ".format(type_adjust),round(adjust_reg,2))
    #     if type_adjust=="stress":
    #         percentile_adjust=-10
    #         adjust_reg=np.sort((df["Real"]-df["Predict"]).dropna().loc[:x_end_train-pd.Timedelta("1D")])[percentile_adjust]
    #         df["Predict_adjust_{}".format(type_adjust)]=df["Predict"]+adjust_reg
    df["mix_regression"]=(df.index.isin(st_msb)*df["Predict_adjust_max"])+\
        ((df.index.isin(ss_day_t0) & (df.index.isin(st_msb)==False))*df["Predict_adjust_seasonal"])+\
        (((df.index.isin(ss_day_t0)==False) & (df.index.isin(st_msb)==False))*df["Predict_adjust_normal"])
    
    
    if per is not None:
        per_filter=per.loc[(per.CURTYP==c) & (per.bank==b)].set_index("Time").loc[:,"WD_{:03}".format(tw)]
        per_filter.values
        df[per_filter.index]=per_filter.values*100
        df["mix_percentile"]=(df.index.isin(st_msb)*df.Stress)+\
        ((df.index.isin(ss_day_t0) & (df.index.isin(st_msb)==False))*df.Seasonal)+\
        (((df.index.isin(ss_day_t0)==False) & (df.index.isin(st_msb)==False))*df.Normal)

    return df

In [ ]:
def plot_reg(df,tw,test_size,b,c,figsize=(10, 4),dpi=100):
    x=df.index
    x_end_train=x[-test_size]
    fig, ax = plt.subplots(1, figsize=figsize,dpi=dpi)
    ax.set_title("Mô hình hồi quy danh mục {} {} Time-window {} ngày".format(b,c,tw).upper())
    ax.grid(alpha=0.3)

    ax.fill_between(x, 0, df["Real"], alpha=0.1,color="gray")
    ax.fill_between(x, 0,df.loc[df.index.isin(covid),"Real"].reindex(x),color="orange", alpha=0.2,label="Covid Period",hatch='.')
    ax.fill_between(x, 0,df.loc[df.index.isin(ss_day_t0),"Real"].reindex(x),color="violet", alpha=0.3,label="Seasonal Period")
    
    ax.plot(x,df["Predict"],color="goldenrod",label="Tỷ lệ Rút hồi quy",alpha=0.7,linewidth=0.7)
    
    ax.plot(x,(df["Predict_adjust_normal"]),"g-",label="Tỷ lệ rút normal regression",alpha=0.7,linewidth=0.7)
    ax.plot(x,(df["Predict_adjust_seasonal"]),"b-",label="Tỷ lệ rút seasonal regression",alpha=0.7,linewidth=0.7)
#     ax.plot(x,(df["Predict_adjust_max"]),color="violet",ls="-",label="Tỷ lệ rút stress regression",alpha=0.7,linewidth=0.7)
    
    ax.plot(x,df["Real"],color="gray",label="Tỷ lệ Rút thực tế")
    ax.plot(x,(df["mix_regression"]),ls="-",color="orangered",label="Tỷ lệ rút hồi quy hiệu chỉnh",linewidth=1.5) 

    ax.legend(ncol=3,loc="upper left")
    ax.vlines(x=x_end_train, ymin=0, ymax=df.max().max()*1.2,ls="--",color="green",linewidth=2)
    ax.annotate(text="Test Period",xy=(x_end_train+pd.Timedelta("3D"),df.max().max()))
#     ax.set_ylim( 0, df.max().max()*1.2)
    ax.set_ylabel("%")
    ax.set_xlabel('Begin date')
    return fig

In [ ]:
def plot_mix(df,tw,test_size,b,c,figsize=(10, 4),dpi=100):
    x=df.index
    x_end_train=x[-test_size]
    fig, ax = plt.subplots(1, figsize=figsize,dpi=dpi)
    ax.set_title("So sánh mô hình hồi quy và percentile danh mục {} {} Time-window {} ngày".format(b,c,tw).upper())
    ax.grid(alpha=0.3)
        
    ax.plot(x,(df["Predict"]),ls="-",color="deepskyblue",label="Tỷ lệ rút hồi quy",alpha=0.7,linewidth=0.7) # olive
    
    ax.plot(x,df["Real"],color="gray",label="Tỷ lệ Rút thực tế")
    ax.fill_between(x, 0, df["Real"], alpha=0.1,color="gray")
    ax.fill_between(x, 0,df.loc[df.index.isin(covid),"Real"].reindex(x),color="orange", alpha=0.1,label="Covid Period",hatch='.')
    ax.fill_between(x, 0,df.loc[df.index.isin(ss_day_t0),"Real"].reindex(x),color="violet", alpha=0.3,label="Seasonal Period")
    
    ax.plot(x,(df["mix_regression"]),ls="-",color="orangered",label="Tỷ lệ rút hồi quy sau hiệu chỉnh",alpha=0.7) # olive
    ax.plot(x,(df["Normal"]),"y--",label="Tỷ lệ rút percentile normal",alpha=0.7)
    ax.plot(x,(df["Seasonal"]),"g--",label="Tỷ lệ rút percentile seasonal",alpha=0.7)
    ax.plot(x,(df["Stress"]),"b--",label="Tỷ lệ rút percentile stress",alpha=0.7)
    
    
    
    ax.legend(ncol=3,loc="upper left")
    ax.vlines(x=x_end_train, ymin=0, ymax=df.max().max()*1.4,ls="--",color="green",linewidth=2)
    ax.annotate(text="Test Period",xy=(x_end_train+pd.Timedelta("3D"),df.max().max()))
    ax.set_ylim( 0, df.max().max()*1.4)
    ax.set_ylabel("%")
    ax.set_xlabel('Begin date')
    return fig


In [ ]:
def performance_cal(df):
    # RMSE chỉ lấy abs, ko có relative
    
    x=[i for i in df.columns if "mix" in i]
    df[[i+"_diff_abs" for i in x]]=df[x].values-df[["Real"]].values
    df[[i+"_diff_rel" for i in x]]=(df[x].values-df[["Real"]].values)/df[["Real"]].values
    df_diff=df[[i for i in df.columns if "diff" in i]]
    performance=pd.concat([(df_diff.pow(2).sum()/len(df_diff)).pow(0.5),
    df_diff.abs().max(),
    df_diff.mean(),
    df_diff.abs().min()],axis=1)
    performance.columns=["RMSE","max_diff","mean_diff","min_diff"]
    performance=performance.T
    for i in performance.columns:
        performance.loc["RMSE",i]=performance.loc["RMSE",i.replace("rel","abs")]
    return performance

In [ ]:
def run_OLS(b,c,tw,wd_lag=avg_wd_lag7,wd_lastmonth=None,ir_names=IR_list[100],ir_delta_booleans=True,macro_names=["GDP"],contractual_booleans=True,dummy_names=["ss_dummy"]
            ,wd_reg=wd_reg,return_to="overview",test_size=test_size,save_fig=False,per=per,figsize=(10, 4),dpi=100):
    y=wd_reg["{}_{}_WD_{:03}".format(c,b,tw)].iloc[:-tw].copy()*100
    const_x=pd.DataFrame([[1]]*len(wd_reg.index),index=wd_reg.index,columns=["const"])
    wd_lag_x=None if (wd_lag is None) else wd_lag[[i for i in wd_lag.columns if "{}_{}_WD_{:03}".format(c,b,tw) in i]].copy()*100
    wd_lastmonth_x=None if (wd_lastmonth is None) else wd_lastmonth[[i for i in wd_lastmonth.columns if "{}_{}_WD_{:03}".format(c,b,tw) in i]].copy()*100
    ir_x=None if ((ir_names is None) | (c=="USD")) else IR_variable[ir_names].copy()
    ir_delta_x = None if ((ir_delta_booleans == False) | (ir_names==None)) else add_IR_delta(ir_x,tw).copy()
    macro_x=None if (macro_names is None) else Macro_variables[macro_names].copy()
    contractual_x=None if (contractual_booleans==False) else contract_rate_variables["{}_{}_contractual_ratio_{}".format(c,b,tw)].copy()*100
    dummy_x=None if (dummy_names is None) else dummy_period_variables[dummy_names].copy()
    x_all=pd.concat([const_x,wd_lag_x,wd_lastmonth_x,ir_x,ir_delta_x,macro_x,contractual_x,dummy_x],axis=1).copy().iloc[:-tw]
    y_train,y_test,x_train,x_test=train_test_split(y,x_all,test_size=test_size,shuffle=False)

    res = sm.OLS(y_train,x_train ).fit()
    predict_y_test = res.predict(x_test)
    RMSE= np.nan if test_size==1 else mean_squared_error(y_test,predict_y_test,squared=False)
#     global df_y
    df_y=pd.concat([y,res.predict(x_all)],axis=1)
    df_y.columns=["Real","Predict"]
#     print("RMSE(%) ",RMSE)
#     print("R2(%) ",res.rsquared*100)
    if return_to=="overview":
        return pd.DataFrame(["{}_{}_WD_{:03}".format(c,b,tw),
                                res.params.index.tolist(),
                                res.params.tolist(),
                                int(sum(res.pvalues>0.05)),
                                res.rsquared,
                                res.f_pvalue,
                                RMSE],
                                index=["Y_feature","X_features","Coef",
                                       "num_unappropriated_pvalue","R_2",
                                       "f_pvalue","RMSE"]).T
    if return_to=="detail":
        print("RMSE trước hiệu chỉnh (%) ",round(RMSE,2))
        print("R2(%) ",round(res.rsquared*100,2))
        print(pd.concat([y,x_all],axis=1).corr()[[y.name]])
        print(res.summary())

    if return_to=="plot_reg":
        df_handled=handling_data_plot(df_y,tw,c=c,b=b,per=None)
        fig=plot_reg(df_handled,tw,test_size,b=b,c=c,figsize=figsize,dpi=dpi)
        if save_fig:
            return fig
        
    if return_to=="plot_mix":
        df_handled=handling_data_plot(df_y,tw,c=c,b=b,per=per)
        fig=plot_mix(df_handled,tw,test_size,b,c,figsize=figsize,dpi=dpi)
        if save_fig:
            global pdf
            pdf.savefig(fig)
            return fig
        
    if return_to=="performance":
        df_handled=handling_data_plot(df_y,tw,c=c,b=b,per=per).iloc[-test_size:]/100
        return performance_cal(df_handled).T.round(4)*100
    
    if return_to=="backtest_mix":
        df_handled=handling_data_plot(df_y,tw,c=c,b=b,per=per).rename(columns={"Real":"WD","mix":"Estimate"})/100
        return df_handled

In [ ]:
b="RB"
c="VND"
tw=30
wd_lag=None # avg_wd_lag7 avg_wd_lag15 avg_wd_lag30 None
wd_lastmonth=  Avg_wd_lag_mark_lastmonth_variables #  # # Last_wd_lag_mark_lastmonth_variables  Avg_wd_lag_mark_lastmonth_variables None
ir_names= ['MSB_G1_Long_Term'] #[  'MSB_G2_Medium_Term', 'MSB_Long_Term']
ir_delta_booleans=False
macro_names=None
contractual_booleans=True
dummy_names=  ["covid_dummy","ss_dummy"] #
# return_to = None                 # detail plot_reg plot_mix backtest_mix
run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
            ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
                ,wd_reg=wd_reg,return_to="backtest_mix",test_size=test_size,save_fig=True,per=per)

#### Run loop all model

In [ ]:
# loop_run_ols for VDN
def loop_run_ols(b,c,list_tw=[7,30,90,180,365]):
    models=pd.DataFrame()
    IR_list_loop=[None] if c=="USD" else IR_list
    number_model=1
    for tw in list_tw:
        for wd_lag_names in [None,avg_wd_lag7,avg_wd_lag15,avg_wd_lag30]:
            for wd_lastmonth_name in [None,Avg_wd_lag_mark_lastmonth_variables,Last_wd_lag_mark_lastmonth_variables]:
                for dummy_names in [None,"ss_dummy","covid_dummy",["ss_dummy","covid_dummy"]]:
                    for contractual_booleans in [False,True]:
                        for ir_names in IR_list_loop:
                            for ir_delta_booleans in ([True] if (ir_names is None) else [False,True]):
                                res=run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag_names,wd_lastmonth=wd_lastmonth_name,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans,
                                            macro_names=None,contractual_booleans=contractual_booleans,dummy_names=dummy_names)
                                models=models.append(res)
                                number_model+=1
                                print(number_model,end="\r")
    models=models[models.num_unappropriated_pvalue==0].copy()
    check_model=models.sort_values(["Y_feature","R_2"],ascending=False).reset_index(drop=True)
#     check_model.to_excel("Regression/check_model_{}_{}.xlsx".format(c,b))
    check_model.to_pickle("Regression/check_model_v5_{}_{}.pickle".format(c,b))
#     return check_model

In [ ]:
for b,c in [("RB_Treated","USD"),("EB","VND"),("SME","VND"),("RB","VND")]:
    loop_run_ols(b,c)

#### Lọc model

In [ ]:
def X_sign(list_x):
    res=[]
    IR_names=IR_variable.columns
    for i in list_x:
        if isinstance(i,(int,float)):
            res.append(np.sign(i))
        elif i =="const":
            res.append(0)
#         elif i in Macro_variables.columns:
#             res.append(0)
        elif i in IR_names:
            res.append(-1)
        elif ("MSB" in i) and ("delta" in i):
            res.append(-1)
        elif i in contract_rate_variables.columns:
            res.append(1)
        elif i in (avg_wd_lag7.columns.tolist()+avg_wd_lag15.columns.tolist()+avg_wd_lag30.columns.tolist()+
                   Last_wd_lag_mark_lastmonth_variables.columns.tolist()+Avg_wd_lag_mark_lastmonth_variables.columns.tolist()):
            res.append(1)
        elif i in "ss_dummy":
            res.append(1)
        else: # covid dummy
            res.append(0)
    return res

def filter_true_model(model):
    l1=model.Coef.map(X_sign).tolist()
    l2=model.X_features.map(X_sign).tolist()
    res=[]
    for i in range(len(l1)):
        check=[(a*b) for a,b in zip(l1[i],l2[i])]
        res.append((-1 not in check))
    return res

def true_model(b,c):
    model=pd.read_pickle("Regression/check_model_v4_{}_{}.pickle".format(c,b))
    res=model.loc[filter_true_model(model)].sort_values(["RMSE","R_2"],ascending=[True,False])
    res.drop_duplicates(['R_2','RMSE'],inplace=True)
    res.to_excel("Regression/true_model_v6_{}_{}.xlsx".format(c,b))
    return res

In [ ]:
for b,c in [("RB_Treated","USD"),("EB","VND"),("SME","VND"),("RB","VND")]:
    true_model(b=b,c=c)

### Check

In [ ]:
b="RB"
c="VND"
tw=30
wd_lag=None # avg_wd_lag7 avg_wd_lag15 avg_wd_lag30 None
wd_lastmonth=  Avg_wd_lag_mark_lastmonth_variables #  # # Last_wd_lag_mark_lastmonth_variables  Avg_wd_lag_mark_lastmonth_variables None
ir_names= ['MSB_G1_Long_Term',] #[  'MSB_G2_Medium_Term', 'MSB_Long_Term']
ir_delta_booleans=False
macro_names=None
contractual_booleans=True
dummy_names=  ["covid_dummy","ss_dummy"] #
return_to = None                 # detail plot_reg plot_mix backtest_mix
run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
            ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
                ,wd_reg=wd_reg,return_to="plot_mix",test_size=test_size,save_fig=True,per=per)
# clear_output()

In [ ]:
run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
            ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
                ,wd_reg=wd_reg,return_to="plot_mix",test_size=test_size,save_fig=True,per=per,type_adjust="normal")
clear_output()

In [ ]:
res_ols=run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
            ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
                ,wd_reg=wd_reg,return_to="plot_reg",test_size=test_size,save_fig=True,per=per,figsize=(20, 6),dpi=100)


### Gap change with regression

In [ ]:
def plot_backtest_mix(b,c,tw,df,MB_Core_all=MB_Core_all):
    df=df.copy()
    df["WDss"]=(df["WD"]*df.index.isin(ss_day_t0)).replace(0,np.nan)
    df["WDst"]=(df["WD"]*df.index.isin(st_msb )).replace(0,np.nan)
    t0=MB_Core_all[[(c,b,"sum_000")]].reindex(df.index).values
    df2=df*t0
    df2["T0"]=t0
    return df2

In [ ]:
per_last_year=pd.read_excel("Parameter_data/FD Percentile 30 2019.xlsx").rename(columns={"CURRENCY_TYPE":"CURTYP"})
per_last_year["Year"]=2019


per_this_year=per[per.bank.isin(["SME","RB_Treated","L"])][['Time','CURTYP','bank',"WD_030"]].replace("RB_Treated","RB")
per_this_year["Year"]=2020


per_gap=per_last_year.append(per_this_year).pivot_table(columns=['CURTYP','bank',"Time","Year"])

per_gap.loc[:,(per_gap.columns.get_level_values("bank")=="L") & (per_gap.columns.get_level_values("Time")!="Stress")]=np.nan

In [ ]:
b="EB"
c="VND"
tw=30
wd_lastmonth=  Last_wd_lag_mark_lastmonth_variables # Last_wd_lag_mark_lastmonth_variables #  # # Last_wd_lag_mark_lastmonth_variables  Avg_wd_lag_mark_lastmonth_variables None
ir_names=  [ 'MSB_12M'] #[  'MSB_G2_12M'] #[ 'MSB_12M'] #None # [ ' 'MSB_12M'', 'MSB_G1_Long_Term'] #[  'MSB_G2_Medium_Term', 'MSB_Long_Term']
contractual_booleans=True
dummy_names= None #[   'ss_dummy']

wd_lag=None # avg_wd_lag7 avg_wd_lag15 avg_wd_lag30 None
ir_delta_booleans=False
macro_names=None
eb_vnd=plot_backtest_mix(b,c,tw,
run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
                ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
                    ,wd_reg=wd_reg,return_to="backtest_mix",test_size=test_size,save_fig=True,per=per)
                 )

b="RB"
c="VND"
tw=30
wd_lastmonth=  Last_wd_lag_mark_lastmonth_variables # Last_wd_lag_mark_lastmonth_variables #  # # Last_wd_lag_mark_lastmonth_variables  Avg_wd_lag_mark_lastmonth_variables None
ir_names=  [ 'MSB_G2_12M'] #[  'MSB_G2_12M'] #[ 'MSB_12M'] #None # [ ' 'MSB_12M'', 'MSB_G1_Long_Term'] #[  'MSB_G2_Medium_Term', 'MSB_Long_Term']
contractual_booleans=True
dummy_names= [ "covid_dummy",  'ss_dummy']

wd_lag=None # avg_wd_lag7 avg_wd_lag15 avg_wd_lag30 None
ir_delta_booleans=False
macro_names=None
rb_vnd=plot_backtest_mix(b,c,tw,
run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
                ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
                    ,wd_reg=wd_reg,return_to="backtest_mix",test_size=test_size,save_fig=True,per=per)
                 )

In [ ]:
def gap_change_data(c="VND",b="RB",per_gap=per_gap,Output_add_core=MB_Core_all,contractual_Result=contractual_Result):
    per_gap_nhcd=per_gap[(c,b)].sort_index(axis=1)
    per_gap_nhcd.columns=["{}_{}".format(x[0],x[1]) for x in per_gap_nhcd.columns]
    df=Output_add_core[[(c,b,"sum_000"),(c,b,"sum_030")]].dropna().iloc[-90:].copy()
    df.columns=["Balance_t0","Balance_tw"]
    df["WD_tw"]=df["Balance_t0"]-df["Balance_tw"]
    df[per_gap_nhcd.columns]=df[["Balance_t0"]].values*per_gap_nhcd.values
    for con in ["Normal","Seasonal","Stress"]:
        a=np.sort([i for i in df.columns if con in i])
        df[con+"_gap_change"]=df[a[1]]-df[a[0]]
    if b=="L":
        df[["Normal_gap_change","Seasonal_gap_change"]]=0
        na_col=df.columns[df.isna().all()].tolist().copy()
        for i in na_col:
            df[i].fillna(contractual_Result[(c,b,"contractual30_0")],inplace=True)
    return df

In [ ]:
def change_gap_type(per_type,reg_type):
    reg_type_2=reg_type.reindex(per_type.index)[["WD","Predict_adjust_normal","Predict_adjust_seasonal"]]\
    .rename(columns={"WD":'WD_tw',"Predict_adjust_normal":"Normal_2020","Predict_adjust_seasonal":"Seasonal_2020"})
    per_type.loc[:,reg_type_2.columns]=reg_type_2
    for con in ["Normal","Seasonal","Stress"]:
        per_type[con+"_gap_change"]=per_type[con+"_2020"]-per_type[con+"_2019"]
    return per_type

In [ ]:
def plot_gap(c,eb_reg=eb_vnd,rb_reg=rb_vnd):
    uni,div_uni=("Millions",1000000) if c=="USD" else ("Billions",1000000000)
    gap=(change_gap_type(per_type=gap_change_data(c="VND",b="RB"),reg_type=rb_reg)+change_gap_type(per_type=gap_change_data(c="VND",b="SME")+gap_change_data(c="VND",b="L"),reg_type=eb_reg)) \
    if c=="VND" else gap_change_data(c=c,b="RB")

    df=gap.iloc[:,2:9]/div_uni
    x=df.index
    fig, axs = plt.subplots(1, 2, figsize=(22, 4),dpi=68)
    ax,ax2 = axs[0],axs[1]

    ax.set_title("DỰ BÁO TỶ LỆ RÚT TIỀN DANH MỤC BWex {}".format(c))
    ax.grid(alpha=0.4)# set gridlines 
    ax.plot(x,df["WD_tw"],color="gray",label="WD thực tế",linewidth=2,alpha=0.7)
    ax.fill_between(x,0,df["WD_tw"],color="gray",alpha=0.2)
    color=["green","orange","brown"]
    m=-1
    for col in df.columns[1:]:
        m+=1
        ax.plot(x,df[col],"-" if m%2==1 else "--",label="WD {}".format(col.replace("_"," ")),alpha=1,linewidth=1.25,color=color[m//2])
    ax.legend(loc="upper left",ncol=4)
    ax.set_ylim(None, df.max().max()*1.8)
    ax.set_xlabel('Begin date')
    ax.set_ylabel(uni)
    
    df=gap.iloc[:,9:]/div_uni
    ax2.set_title("DỰ BÁO TỶ LỆ RÚT TIỀN THAY ĐỔI ẢNH HƯỞNG ĐẾN GAP DANH MỤC BWex {}".format(c))
    plt.grid(alpha=0.4)
    m=-1
    for col in df.columns:
        m+=1
        ax2.plot(x,df[col],color=color[m],label=col)
    ax2.legend(loc="upper left",ncol=3)
    ax2.set_ylim(None, df.max().max()*1.2+ abs(df.min().min()))
    ax2.set_xlabel('Begin date')
    ax2.set_ylabel(uni)
    plt.show()
    return fig

In [ ]:
for c in ["VND","USD"]:
    plot_gap(c)#.savefig("Plot_output/gap_30_{}_reg.png".format(c),bbox_inches="tight")

### Backtest mix kết hợp percentile + hồi quy

In [ ]:
def plot_backtest_mix(b,c,tw,df,MB_Core_all=MB_Core_all):
    df=df.copy()
    df["WDss"]=(df["WD"]*df.index.isin(ss_day_t0)).replace(0,np.nan)
    df["WDst"]=(df["WD"]*df.index.isin(st_msb )).replace(0,np.nan)
    t0=MB_Core_all[[(c,b,"sum_000")]].reindex(df.index).values
    df2=df*t0
    df2["T0"]=t0
    return df2

In [ ]:
b="EB"
c="VND"
tw=30
wd_lastmonth=  Last_wd_lag_mark_lastmonth_variables # Last_wd_lag_mark_lastmonth_variables #  # # Last_wd_lag_mark_lastmonth_variables  Avg_wd_lag_mark_lastmonth_variables None
ir_names=  [ 'MSB_12M'] #[  'MSB_G2_12M'] #[ 'MSB_12M'] #None # [ ' 'MSB_12M'', 'MSB_G1_Long_Term'] #[  'MSB_G2_Medium_Term', 'MSB_Long_Term']
contractual_booleans=True
dummy_names= None #[   'ss_dummy']

wd_lag=None # avg_wd_lag7 avg_wd_lag15 avg_wd_lag30 None
ir_delta_booleans=False
macro_names=None
eb_vnd=plot_backtest_mix(b,c,tw,
run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
                ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
                    ,wd_reg=wd_reg,return_to="backtest_mix",test_size=test_size,save_fig=True,per=per)
                 )

b="RB"
c="VND"
tw=30
wd_lastmonth=  Last_wd_lag_mark_lastmonth_variables # Last_wd_lag_mark_lastmonth_variables #  # # Last_wd_lag_mark_lastmonth_variables  Avg_wd_lag_mark_lastmonth_variables None
ir_names=  [ 'MSB_G2_12M'] #[  'MSB_G2_12M'] #[ 'MSB_12M'] #None # [ ' 'MSB_12M'', 'MSB_G1_Long_Term'] #[  'MSB_G2_Medium_Term', 'MSB_Long_Term']
contractual_booleans=True
dummy_names= [ "covid_dummy",  'ss_dummy']

wd_lag=None # avg_wd_lag7 avg_wd_lag15 avg_wd_lag30 None
ir_delta_booleans=False
macro_names=None
rb_vnd=plot_backtest_mix(b,c,tw,
run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
                ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
                    ,wd_reg=wd_reg,return_to="backtest_mix",test_size=test_size,save_fig=True,per=per)
                 )

In [ ]:
def backtest_processing_data(c="VND",b="EB",tw=30,per=per,Output=MB_Core_all):
    per_nhcd=per.loc[(per.bank==b) & (per.CURTYP==c),["Time","WD_{:03}".format(tw)]].set_index("Time").sort_index()
    df=Output[(c,b)][["sum_{:03}".format(i) for i in [0,tw]]]
    df.columns=["T0","TW"]
    df["WD"]=df["T0"]-df["TW"]
    df[per_nhcd.index]=(df["T0"].values*per_nhcd.values).T
    df["WDss"]=(df["WD"]*df.index.isin(ss_day_t0)).replace(0,np.nan)
    df["WDst"]=(df["WD"]*df.index.isin(st_msb )).replace(0,np.nan)
    
    #contractual
    df_contractual=contractual_Result[(c,b,"contractual{}_0".format(tw))].copy()
    df_contractual.columns=["Contractual_WD"]
    df["Contractual_WD"]=np.nan
    df["Contractual_WD"].fillna(df_contractual,inplace=True)
    
    if b=="L":
        df["Normal"]=df["Contractual_WD"]
        df["Seasonal"]=df["Contractual_WD"]
        
    return df.iloc[:-tw]

In [ ]:
L_VND=backtest_processing_data(c="VND",b="L")
L_VND["Normal"]=L_VND["Seasonal"]=L_VND["Contractual_WD"]
eb_vnd_per_ts=backtest_processing_data(c="VND",b="SME")+L_VND
rb_vnd_per_ts= backtest_processing_data(c="VND",b="RB")
bwex_vnd_per_ts=rb_vnd_per_ts+eb_vnd_per_ts

In [ ]:
def concat_data_plot(per,reg):
    return pd.concat([per,reg[[i for i in reg.columns if i not in per.columns]]],axis=1)

In [ ]:
def Backtest_Timeseries_mix(b,c,tw,df):
    
    uni,div_uni=("Millions",1000000) if c=="USD" else ("Billions",1000000000)
    df=df/div_uni
    max_y_axis=df.drop(["T0","TW","Contractual_WD"],axis=1).max().max()*1.3
    x = df.index

    
    # plot time count cif
    fig, ax1 = plt.subplots(1,1, figsize=(18, 10),dpi=100)

    ax1.set_title("BACKTEST RÚT TIỀN DANH MỤC {} {} TIME-WINDOW {} ngày".format(b,c,tw))

    ax1.grid(alpha=0.2)
    
    cols=["WD","WDss","WDst","Normal","Seasonal","Stress","mix_regression"]
    col_labels=["Giá trị rút tại tại {} ngày sau tại thời điểm T0".format(tw),
               "Giá trị rút tại tại {} ngày sau tại thời điểm T0 là giai đoạn mùa vụ".format(tw),
                "Giá trị rút tại tại {} ngày sau tại thời điểm T0 là giai đoạn căng thẳng".format(tw),
                "Ước lượng percentile rút thông thường","Ước lượng percentile rút mùa vụ","Ước lượng percentile rút căng thẳng",
                "Ước lượng hồi quy sau hiệu chỉnh",
               ]
    Colors=["grey","green","orange","brown","blue","olivedrab","red"]
    
    for co,la,color in zip(cols,col_labels,Colors):
        ls,alp=("-",0.7) if co in ["WD", "WDss","WDst","mix_regression"] else ("--",1)
        li=1.5 if co=="mix_regression" else 1
        try:
            ax1.plot(x,gaussian_filter1d(df[co], sigma=1),label=la,color=color,ls=ls,alpha=alp,linewidth=li)

        except:
            print("lỗi ",co)
            pass
    ax1.fill_between(x, 0, gaussian_filter1d(df["Contractual_WD"], sigma=1),label="Giá trị hợp đồng đáo hạn trong timewindow",color="violet", alpha=0.1)
    ax1.fill_between(x, 0, gaussian_filter1d(df["WD"], sigma=1), alpha=0.2)


    #plot normal period
    end_train_day_tw=end_train_day-pd.Timedelta(days=tw)
    ax1.vlines(x=x[(x<=end_train_day_tw) & (x.isin(ss_day_t0)==False)][-365], ymin=0, ymax=max_y_axis,ls="--")
    ax1.vlines(x=x[(x<=end_train_day_tw) & (x.isin(ss_day_t0)==False)][-1], ymin=0, ymax=max_y_axis,ls="--")
    ax1.annotate(text="Normal Period",xy=(x[x.isin(ss_day_t0)==False][-200],max_y_axis*0.8))
    
    ax1.legend(loc="upper left",ncol=3)
    ax1.set_ylim(0, max_y_axis)

    ax1.set_ylabel(uni)

    ax1.set_xlabel('Begin date')
    dayloc = mdates.MonthLocator([3,6,9,12])
    ax1.xaxis.set_major_locator(dayloc)
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    fig.autofmt_xdate()
#     plt.grid(alpha=0.4)
    plt.show()
    return fig

In [ ]:
def plot_backtest_violation_reg(df,b,c,tw):
    
    df=df[df.index<=(end_train_day-pd.Timedelta(days=tw))].rename(columns={"mix_regression":"Estimate"}).copy()
    uni,div_uni=("Millions",1000000) if c=="USD" else ("Billions",1000000000)
    list_fig=[]
    list_condition=["All"] if b=="L" else ["Normal","Seasonal","All"]
    for condition in list_condition:
        fig, axs = plt.subplots(1, 2, figsize=(22.5, 3),dpi=68)
        if condition=="Normal":
            df2=df[df.index.isin(ss_day_t0)==False].iloc[-365:].copy()
        elif condition=="Seasonal":
            df2=df[df.index.isin(ss_day_t0)].copy()
        else:
            df2 = df.copy()
            
        df2["number_vio"]=(df2["WD"]>df2["Estimate"]).astype(int)
        df2["amount_vio"]=(df2["WD"]-df2["Estimate"])*df2["number_vio"]
        df2["per_amount_vio"]=df2["amount_vio"]/df2["T0"]
    #         global df2_month
        df2_month=df2.groupby(pd.Grouper(freq='MS')).agg({"number_vio":"sum","amount_vio":"mean","per_amount_vio":["max","mean"]}).applymap(lambda x: x if x>0 else np.nan)
        df2_month.columns=df2_month.columns.map(lambda x: x[0]+"_"+x[1])
        x=df2_month.index

        ax_1,ax_2 = axs[0],axs[1]
        #count violation
        ax_1.grid(alpha=0.4)
        ax_1.bar(x,df2_month.number_vio_sum.fillna(0),width=15)
        ax_1.set_ylabel('Số vi phạm')
        ax_1.set_title("BACKTEST SỐ LẦN VI PHẠM ƯỚC LƯỢNG HỒI QUY DANH MỤC {} {} TIME-WINDOW {} NGÀY ({})".format(b,c,tw,condition))
        for i in df2_month.dropna().index:
            ax_1.annotate(text=df2_month.loc[i,"number_vio_sum"].astype(int),xy=(i,df2_month.loc[i,"number_vio_sum"]),  va='center',xytext=(-3,7), textcoords='offset points')

        dayloc = mdates.MonthLocator([3,6,9,12])
        ax_1.xaxis.set_major_locator(dayloc)
        ax_1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        try:
            ax_1.set_ylim(0, df2_month.number_vio_sum.max()*1.2)
        except:
            pass
        ax_1.set_xlabel('Begin date')

        # balance of violation
        ax_2.set_title("BACKTEST MỨC ĐỘ VI PHẠM ƯỚC LƯỢNG HỒI QUY DANH MỤC {} {} TIME-WINDOW {} NGÀY ({})".format(b,c,tw,condition))
        ax_2.grid(alpha=0.4)
        ax_2.fill_between(x, 0, df2_month['per_amount_vio_max'].fillna(0)*100, alpha=0.4,color="orange",label="Tỷ lệ (%) vi phạm lớn nhất")
        ax_2.fill_between(x, 0, df2_month['per_amount_vio_mean'].fillna(0)*100, alpha=1,color="orange",label="Tỷ lệ (%) vi phạm trung bình")
        ax_2.set_ylabel('%')
        ax_2.set_xlabel('Begin date')

        ax2_2=ax_2.twinx()
        ax2_2.bar(x,df2_month["amount_vio_mean"].fillna(0)/div_uni,width=15,label="Giá trị vi phạm trung bình")
        ax2_2.set_ylabel(uni)

        ax_2.legend(loc="upper left",ncol=2)
        ax2_2.legend(loc="upper right")

        ax_2.xaxis.set_major_locator(dayloc)
        ax_2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
        try:
            ax_2.set_ylim(0, df2_month['per_amount_vio_max'].max()*140)
            ax2_2.set_ylim(0, df2_month['amount_vio_mean'].max()/div_uni*1.4)
        except:
            pass
        fig.autofmt_xdate()
        plt.show()
        list_fig.append(fig)
    return list_fig,list_condition

In [ ]:
Backtest_Timeseries_mix(df=concat_data_plot(rb_vnd_per_ts,rb_vnd),b="RB",c="VND",tw=30)#.savefig("Regression/Backtest_Timeseries_mix_{}_{}_{}_v2.png".format("RB","VND",30),bbox_inches="tight")
Backtest_Timeseries_mix(df=concat_data_plot(eb_vnd_per_ts,eb_vnd),b="EB",c="VND",tw=30)#.savefig("Regression/Backtest_Timeseries_mix_{}_{}_{}_v2.png".format("EB","VND",30),bbox_inches="tight")
Backtest_Timeseries_mix(df=concat_data_plot(bwex_vnd_per_ts,eb_vnd+rb_vnd),b="BWex",c="VND",tw=30)#.savefig("Regression/Backtest_Timeseries_mix_{}_{}_{}_v2.png".format("BWex","VND",30),bbox_inches="tight")

### Export

In [ ]:
def export_reg(b,c,tw,wd_lag ,wd_lastmonth ,ir_names,ir_delta_booleans,macro_names,contractual_booleans,dummy_names):
    for return_to in ['performance',"plot_reg","plot_mix","backtest_mix"]:
        res=run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
            ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
                ,wd_reg=wd_reg,return_to=return_to,test_size=test_size,save_fig=True,per=per)
        if return_to=='performance':
            res.to_excel("Regression/performance_{}_{}_{}.xlsx".format(b,c,tw))
#         if return_to=='plot_reg':
#             res.savefig("Regression/plot_reg_{}_{}_{}.png".format(b,c,tw),bbox_inches="tight")
        if return_to=='plot_mix':
            res.savefig("Regression/plot_mix_{}_{}_{}.png".format(b,c,tw),bbox_inches="tight")
        if return_to=='backtest_mix':
            df2 = plot_backtest_mix(b,c,tw,res)
            Backtest_Timeseries_mix(b,c,tw,df2).savefig("Regression/Backtest_Timeseries_mix_{}_{}_{}.png".format(b,c,tw),bbox_inches="tight")
            list_fig,list_condition=plot_backtest_violation_reg(df2,b,c,tw)
            for fig,condition in zip(list_fig,list_condition):
                fig.savefig("Regression/plot_backtest_violation_reg_{}_{}_{}_{}.png".format(b,c,tw,condition),bbox_inches="tight")
            

In [ ]:
# RB VND 7 ('VND_RB_WD_007_last_lastmonth', 'MSB_G1_1M', 'MSB_G2_12M', 'VND_RB_contractual_ratio_7', 'ss_dummy', 'covid_dummy')
# RB VND 30 ( 'VND_RB_WD_030_last_lastmonth', 'MSB_G2_12M', 'VND_RB_contractual_ratio_30', 'ss_dummy', 'covid_dummy')
# RB VND 90 ('VND_RB_WD_090_last_lastmonth', 'MSB_G2_12M', 'covid_dummy')
# RB VND 180 ('VND_RB_WD_180_last_lastmonth')

# VND_EB_WD_007	('VND_EB_WD_007_last_lastmonth', 'MSB_1M', 'MSB_G1_12M', 'VND_EB_contractual_ratio_7', 'ss_dummy')
# VND_EB_WD_030	('VND_EB_WD_030_last_lastmonth', 'MSB_12M', 'VND_EB_contractual_ratio_30')
# VND_EB_WD_090	 ('VND_EB_WD_090_last_lastmonth', 'VND_EB_contractual_ratio_90')
# VND_EB_WD_180	 ('VND_EB_WD_180_last_lastmonth')

# USD_RB_Treated_WD_090	 ('USD_RB_Treated_WD_090_last_lastmonth')
# USD_RB_Treated_WD_180	 ('USD_RB_Treated_WD_180_last_lastmonth')



In [ ]:
list_model=[
    ("RB","VND",7,Last_wd_lag_mark_lastmonth_variables,['MSB_G1_1M', 'MSB_G2_12M'],True, [ "covid_dummy",  'ss_dummy']),
    ("RB","VND",30,Last_wd_lag_mark_lastmonth_variables,[ 'MSB_G2_12M'],True, [ "covid_dummy",  'ss_dummy']),
    ("RB","VND",90,Last_wd_lag_mark_lastmonth_variables,[ 'MSB_G2_12M'],False, [ "covid_dummy"]),
    ("RB","VND",180,Last_wd_lag_mark_lastmonth_variables,None,False, None),
    ("EB","VND",7,Last_wd_lag_mark_lastmonth_variables,['MSB_1M', 'MSB_G1_12M'],True, ['ss_dummy']),
    ("EB","VND",30,Last_wd_lag_mark_lastmonth_variables,['MSB_12M'],True, None),
    ("EB","VND",90,Last_wd_lag_mark_lastmonth_variables,None,True, None),
    ("EB","VND",180,Last_wd_lag_mark_lastmonth_variables,None,False,None),
    ("RB_Treated","USD",90,Last_wd_lag_mark_lastmonth_variables,None,False,None),
    ("RB_Treated","USD",180,Last_wd_lag_mark_lastmonth_variables,None,False,None),
]

In [ ]:
wd_lag=None 
ir_delta_booleans=False
macro_names=None
df_res=pd.DataFrame()
for b,c,tw,wd_lastmonth,ir_names,contractual_booleans,dummy_names in list_model:
    df=run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
                ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
                    ,wd_reg=wd_reg,return_to="overview",test_size=test_size,save_fig=True,per=per,dpi=200,figsize=(20,8))#.savefig("Regression/plot_mix_{}_{}_{}_high_quantity.png".format(b,c,tw),bbox_inches="tight")
    df_res=df_res.append(df)


df_res.to_excel("res_v2.xlsx")

In [ ]:
wd_lag=None 
ir_delta_booleans=False
macro_names=None
pdf=PdfPages('kq hoi quy v3 FD LR.pdf')
for b,c,tw,wd_lastmonth,ir_names,contractual_booleans,dummy_names in list_model:
    a=run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
                ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
                    ,wd_reg=wd_reg,return_to="plot_mix",test_size=test_size,save_fig=True,per=per,dpi=200,figsize=(20,8))#.savefig("Regression/plot_mix_{}_{}_{}_high_quantity.png".format(b,c,tw),bbox_inches="tight")
    pdf.savefig(a)
pdf.close()

In [ ]:
b="RB"
c="VND"
tw=7
wd_lastmonth=  Last_wd_lag_mark_lastmonth_variables # Last_wd_lag_mark_lastmonth_variables #  # # Last_wd_lag_mark_lastmonth_variables  Avg_wd_lag_mark_lastmonth_variables None
ir_names=  [ 'MSB_G1_1M', 'MSB_G2_12M'] #[  'MSB_G2_12M'] #[ 'MSB_12M'] #None # [ ' 'MSB_12M'', 'MSB_G1_Long_Term'] #[  'MSB_G2_Medium_Term', 'MSB_Long_Term']
contractual_booleans=True
dummy_names= [ "covid_dummy",  'ss_dummy']

wd_lag=None 
ir_delta_booleans=False
macro_names=None

run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
                ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
                    ,wd_reg=wd_reg,return_to="plot_mix",test_size=test_size,save_fig=True,per=per,dpi=200,figsize=(20,8))#.savefig("Regression/plot_mix_{}_{}_{}_high_quantity.png".format(b,c,tw),bbox_inches="tight")


In [ ]:
b="RB"
c="VND"
tw=30
wd_lastmonth=  Last_wd_lag_mark_lastmonth_variables # Last_wd_lag_mark_lastmonth_variables #  # # Last_wd_lag_mark_lastmonth_variables  Avg_wd_lag_mark_lastmonth_variables None
ir_names=  [ 'MSB_G2_12M'] #[  'MSB_G2_12M'] #[ 'MSB_12M'] #None # [ ' 'MSB_12M'', 'MSB_G1_Long_Term'] #[  'MSB_G2_Medium_Term', 'MSB_Long_Term']
contractual_booleans=True
dummy_names= [ "covid_dummy",  'ss_dummy']

wd_lag=None # avg_wd_lag7 avg_wd_lag15 avg_wd_lag30 None
ir_delta_booleans=False
macro_names=None
rb=plot_backtest_mix(b,c,tw,
run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
                ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
                    ,wd_reg=wd_reg,return_to="backtest_mix",test_size=test_size,save_fig=True,per=per)
                 )

In [ ]:
b="EB"
c="VND"
tw=30
wd_lastmonth=  Last_wd_lag_mark_lastmonth_variables # Last_wd_lag_mark_lastmonth_variables #  # # Last_wd_lag_mark_lastmonth_variables  Avg_wd_lag_mark_lastmonth_variables None
ir_names=  [ 'MSB_12M'] #[  'MSB_G2_12M'] #[ 'MSB_12M'] #None # [ ' 'MSB_12M'', 'MSB_G1_Long_Term'] #[  'MSB_G2_Medium_Term', 'MSB_Long_Term']
contractual_booleans=True
dummy_names= None #[   'ss_dummy']

wd_lag=None # avg_wd_lag7 avg_wd_lag15 avg_wd_lag30 None
ir_delta_booleans=False
macro_names=None
eb_vnd=plot_backtest_mix(b,c,tw,
run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
                ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
                    ,wd_reg=wd_reg,return_to="backtest_mix",test_size=test_size,save_fig=True,per=per)
                 )

b="RB"
c="VND"
tw=30
wd_lastmonth=  Last_wd_lag_mark_lastmonth_variables # Last_wd_lag_mark_lastmonth_variables #  # # Last_wd_lag_mark_lastmonth_variables  Avg_wd_lag_mark_lastmonth_variables None
ir_names=  [ 'MSB_G2_12M'] #[  'MSB_G2_12M'] #[ 'MSB_12M'] #None # [ ' 'MSB_12M'', 'MSB_G1_Long_Term'] #[  'MSB_G2_Medium_Term', 'MSB_Long_Term']
contractual_booleans=True
dummy_names= [ "covid_dummy",  'ss_dummy']

wd_lag=None # avg_wd_lag7 avg_wd_lag15 avg_wd_lag30 None
ir_delta_booleans=False
macro_names=None
rb_vnd=plot_backtest_mix(b,c,tw,
run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
                ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
                    ,wd_reg=wd_reg,return_to="backtest_mix",test_size=test_size,save_fig=True,per=per)
                 )

In [ ]:
bwex_vnd = eb_vnd+rb_vnd
Backtest_Timeseries_mix("BWex",c,tw,bwex_vnd).savefig("Regression/Backtest_Timeseries_mix_{}_{}_{}.png".format("BWex",c,tw),bbox_inches="tight")
list_fig,list_condition=plot_backtest_violation_reg(bwex_vnd,"BWex",c,tw)
for fig,condition in zip(list_fig,list_condition):
    fig#.savefig("Regression/plot_backtest_violation_reg_{}_{}_{}_{}.png".format("BWex",c,tw,condition),bbox_inches="tight")

In [ ]:
b="EB"
c="VND"
tw=30
wd_lastmonth=  Last_wd_lag_mark_lastmonth_variables # Last_wd_lag_mark_lastmonth_variables #  # # Last_wd_lag_mark_lastmonth_variables  Avg_wd_lag_mark_lastmonth_variables None
ir_names=  [ 'MSB_12M'] #[  'MSB_G2_12M'] #[ 'MSB_12M'] #None # [ ' 'MSB_12M'', 'MSB_G1_Long_Term'] #[  'MSB_G2_Medium_Term', 'MSB_Long_Term']
contractual_booleans=True
dummy_names= None #[   'ss_dummy']

wd_lag=None # avg_wd_lag7 avg_wd_lag15 avg_wd_lag30 None
ir_delta_booleans=False
macro_names=None
plot_backtest_mix(b,c,tw,
eb=run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
                ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
                    ,wd_reg=wd_reg,return_to="backtest_mix",test_size=test_size,save_fig=True,per=per)
                 )
# export_reg(b,c,tw,wd_lag ,wd_lastmonth ,ir_names,ir_delta_booleans,macro_names,contractual_booleans,dummy_names)
# clear_output()
# for return_to in ["detail","performance"]:
#     print(run_OLS(b=b,c=c,tw=tw,wd_lag=wd_lag,wd_lastmonth=wd_lastmonth,ir_names=ir_names,ir_delta_booleans=ir_delta_booleans
#                 ,macro_names=macro_names,contractual_booleans=contractual_booleans,dummy_names=dummy_names
#                     ,wd_reg=wd_reg,return_to=return_to,test_size=test_size,save_fig=True,per=per))